# Anchor tutorial

#### Sections
* [Anchors](#Anchors)
* [`anchor` "open"](#anchor-"open")
    * [Interrogating indices with `.pt` accessor](#Interrogating-indices-with-.pt-accessor)
        * [Trading status](#Trading-status)
        * [Number of trading minutes](#Number-of-trading-minutes)
        * [Intervals' lengths](#Intervals'-lengths)
    * [`force`](#force)
        * [Force to period end with `openend`](#Force-to-period-end-with-openend)
    * [Indices overlapping next session](#Indices-overlapping-next-session)
* [`anchor` "workback"](#anchor-"workback")
* [Multiple calendars](#Multiple-calendars)
    * [`anchor` "open" with multiple calendars](#anchor-"open"-with-multiple-calendars)
        * [`.pt` accessor methods](#.pt-accessor-methods)
        * [Calendars with breaks](#Calendars-with-breaks)
            * [Data source does not observe the break](#Data-source-does-not-observe-the-break)
            * [Indices of trading indexes of different calendars overlap](#Indices-of-trading-indexes-of-different-calendars-overlap)
    * [`anchor` "workback" with multiple calendars](#anchor-"workback"-with-multiple-calendars)
    * [`openend`](#openend)
* [Daily prices](#Daily-prices)
* [Takeaways](#Takeaways)

#### Notes
* The cell **outputs** shown in this tutorial are based on executing the cells at **2022-05-13 09:39 UTC** (05:39 New York). Simply rerun the cells to bring any dynamic output up to date.

## Setup

Run the following cell to import tutorial dependencies.

In [2]:
from market_prices import PricesYahoo
import pandas as pd
from market_prices.support import tutorial_helpers as th

Run the following cell to define values used in the first part of this tutorial.

In [3]:
_prices_mix = PricesYahoo("MSFT, 9988.HK, AZN.L")
xnys = _prices_mix.calendars["MSFT"]
xhkg = _prices_mix.calendars["9988.HK"]
xlon = _prices_mix.calendars["AZN.L"]
_calendars = [xnys, xhkg, xlon]
_session_length = [
    pd.Timedelta(hours=6, minutes=30),
    pd.Timedelta(hours=6, minutes=30),
    pd.Timedelta(hours=8, minutes=30),
]
# get sessions for which price data available at all base intervals
_sessions_range = th.get_sessions_range_for_bi(
    _prices_mix, _prices_mix.bis.T1
)
start_session, end_session = th.get_conforming_sessions(
    _calendars, _session_length, *_sessions_range, 2
)

# get sessions for which intraday price data available only at H1 base iterval.
_sessions_range = th.get_sessions_range_for_bi(
    _prices_mix, _prices_mix.bis.H1
)
start_session_h1, end_session_h1 = th.get_conforming_sessions(
    _calendars, _session_length, *_sessions_range, 2
)
session_h1 = end_session_h1

## Anchors
The [intervals tutorial](./intervals.ipynb) explained how intraday price data is served as a table with each indice representing an interval of time. There are no gaps between the indices during any period when an exchange associated with any symbol is open. But what should be the origin from which the indices are evaluated? Or put another way, where should the indices be anchored?

This tutorial:
* explains the **`anchor`** options:
    * **"open"** to anchor indices on each session's open (all indices have the same length).
    * **"workback"** to anchor an indice on the period end and work back (all indices cover the same number of 'trading minutes').
* shows the use of **`.pt accessor`** methods to interrogate whether indices cover times when an exchange is open, closed or both.
* shows how the **`force`** option can be used to curtail indices to reflect only trading periods.
* shows how the **`openend`** option determines the final indice when the period end is an unaligned session close.

## `anchor` "open"
By default `market_prices` anchors data on each session open. This behaviour can be explicitly requested by passing the `get` method's  `anchor` parameter as "open".

In [4]:
prices = PricesYahoo("MSFT")  # Prices for Microsoft's NYSE listing.
print(f"{start_session=}\n{end_session=}\n")  # for reference

df_30 = prices.get("30T", start_session, end_session, anchor="open")
df_30

start_session=Timestamp('2022-04-19 00:00:00')
end_session=Timestamp('2022-04-20 00:00:00')



symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 09:30:00, 2022-04-19 10:00:00)  279.720001  282.420013   
[2022-04-19 10:00:00, 2022-04-19 10:30:00)  282.399994  284.369385   
[2022-04-19 10:30:00, 2022-04-19 11:00:00)  283.359985  284.750000   
[2022-04-19 11:00:00, 2022-04-19 11:30:00)  283.880005  284.662201   
[2022-04-19 11:30:00, 2022-04-19 12:00:00)  284.410004  284.820007   
[2022-04-19 12:00:00, 2022-04-19 12:30:00)  284.269989  284.269989   
[2022-04-19 12:30:00, 2022-04-19 13:00:00)  283.600006  283.989990   
[2022-04-19 13:00:00, 2022-04-19 13:30:00)  283.780304  284.059998   
[2022-04-19 13:30:00, 2022-04-19 14:00:00)  283.179993  284.019989   
[2022-04-19 14:00:00, 2022-04-19 14:30:00)  283.545013  283.839996   
[2022-04-19 14:30:00, 2022-04-19 15:00:00)  283.519989  284.359985   
[2022-04-19 15:00:00, 2022-04-19 15:30:00)  284.279999  285.582092   
[2022-04-19 15:30:00, 2022-04-19 16:00:00)  285.279999  286.170013   
[2022-04-20 09:30:00, 2022-04-20 10:00:00)  289.399994  289.700012   
[2022-04-20 10:00:00, 2022-04-20 10:30:00)  286.380005  288.390015   
[2022-04-20 10:30:00, 2022-04-20 11:00:00)  288.149994  288.890015   
[2022-04-20 11:00:00, 2022-04-20 11:30:00)  288.079987  288.130005   
[2022-04-20 11:30:00, 2022-04-20 12:00:00)  286.519989  287.600006   
[2022-04-20 12:00:00, 2022-04-20 12:30:00)  287.209991  287.380005   
[2022-04-20 12:30:00, 2022-04-20 13:00:00)  285.740997  287.239990   
[2022-04-20 13:00:00, 2022-04-20 13:30:00)  286.459991  287.369995   
[2022-04-20 13:30:00, 2022-04-20 14:00:00)  286.880005  288.010010   
[2022-04-20 14:00:00, 2022-04-20 14:30:00)  287.549988  287.980011   
[2022-04-20 14:30:00, 2022-04-20 15:00:00)  286.959991  287.183014   
[2022-04-20 15:00:00, 2022-04-20 15:30:00)  286.100006  287.399994   
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  287.109985  288.119904   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 09:30:00, 2022-04-19 10:00:00)  278.410004  282.420013  3538466.0  
[2022-04-19 10:00:00, 2022-04-19 10:30:00)  282.059998  283.329987  1738630.0  
[2022-04-19 10:30:00, 2022-04-19 11:00:00)  283.230011  283.929993  1274094.0  
[2022-04-19 11:00:00, 2022-04-19 11:30:00)  283.600006  284.420013  1236148.0  
[2022-04-19 11:30:00, 2022-04-19 12:00:00)  283.899994  284.359985   838443.0  
[2022-04-19 12:00:00, 2022-04-19 12:30:00)  283.519989  283.587799   699286.0  
[2022-04-19 12:30:00, 2022-04-19 13:00:00)  283.290009  283.809998   810317.0  
[2022-04-19 13:00:00, 2022-04-19 13:30:00)  283.149994  283.200012   788386.0  
[2022-04-19 13:30:00, 2022-04-19 14:00:00)  282.980011  283.549988  1007906.0  
[2022-04-19 14:00:00, 2022-04-19 14:30:00)  282.910004  283.519989   967876.0  
[2022-04-19 14:30:00, 2022-04-19 15:00:00)  283.179993  284.260010   878248.0  
[2022-04-19 15:00:00, 2022-04-19 15:30:00)  284.049988  285.250000  1281596.0  
[2022-04-19 15:30:00, 2022-04-19 16:00:00)  284.500000  285.380005  3282886.0  
[2022-04-20 09:30:00, 2022-04-20 10:00:00)  285.980011  286.399994  3810859.0  
[2022-04-20 10:00:00, 2022-04-20 10:30:00)  285.799988  288.100006  1912616.0  
[2022-04-20 10:30:00, 2022-04-20 11:00:00)  287.589996  288.084991  1198312.0  
[2022-04-20 11:00:00, 2022-04-20 11:30:00)  286.220001  286.510010  1423247.0  
[2022-04-20 11:30:00, 2022-04-20 12:00:00)  286.339996  287.170013  1098033.0  
[2022-04-20 12:00:00, 2022-04-20 12:30:00)  285.370209  285.750000  1045505.0  
[2022-04-20 12:30:00, 2022-04-20 13:00:00)  285.679993  286.480011   834098.0  
[2022-04-20 13:00:00, 2022-04-20 13:30:00)  286.220001  286.929993   810220.0  
[2022-04-20 13:30:00, 2022-04-20 14:00:00)  286.750000  287.570007   789662.0  
[2022-04-20 14:00:00, 2022-04-20 14:30:00)  286.929993  286.950012   823393.0  
[2022-04-20 14:30:00, 2022-04-20 15:00:

The table offer prices for the stock Microsoft over two consecutive sessions at 30 minute intervals. The first indice begins on the first session's open, at 09.30 (local time). The intervals are then contiguous through to the close of the first session at 16.00. The next indice then offers the first interval of the second session, begining on the second session's 09.30 open. The following indices are then again contiguous through to the second session's 16.00 close.

The NYSE does not have a session break, although some exchanges pause trading over lunch. This effectively splits each session into a morning (am) and an afternoon (pm) subsession. In this case anchoring "open" has the effect of anchoring am subsession indices to the am open and anchoring the pm subsession indices to the pm open, as if they were separate sessions. (See [Calendars with breaks](#Calendars-with-breaks) section for some specific cases where indices covering afternoon subsessions are necessarily anchored to the morning open.)

Consider the following prices for Alibaba's lising on the Hong Kong exchange (symbol '9988.HK').

In [5]:
prices_hk = PricesYahoo("9988.HK")
session = end_session
prices_hk.get("30T", session, session, anchor="open")

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:30:00, 2022-04-20 10:00:00)  91.199997  91.900002  90.599998   
[2022-04-20 10:00:00, 2022-04-20 10:30:00)  91.349998  91.500000  90.750000   
[2022-04-20 10:30:00, 2022-04-20 11:00:00)  90.800003  91.250000  90.500000   
[2022-04-20 11:00:00, 2022-04-20 11:30:00)  91.099998  92.199997  91.050003   
[2022-04-20 11:30:00, 2022-04-20 12:00:00)  92.000000  92.949997  91.949997   
[2022-04-20 13:00:00, 2022-04-20 13:30:00)  92.650002  92.750000  91.500000   
[2022-04-20 13:30:00, 2022-04-20 14:00:00)  91.699997  91.900002  91.050003   
[2022-04-20 14:00:00, 2022-04-20 14:30:00)  91.150002  91.349998  90.550003   
[2022-04-20 14:30:00, 2022-04-20 15:00:00)  90.500000  91.050003  90.300003   
[2022-04-20 15:00:00, 2022-04-20 15:30:00)  90.650002  91.150002  90.550003   
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  91.099998  91.300003  90.599998   

symbol                                                            
                                                close     volume  
[2022-04-20 09:30:00, 2022-04-20 10:00:00)  91.349998  1901362.0  
[2022-04-20 10:00:00, 2022-04-20 10:30:00)  90.800003  1360453.0  
[2022-04-20 10:30:00, 2022-04-20 11:00:00)  91.099998  1590994.0  
[2022-04-20 11:00:00, 2022-04-20 11:30:00)  92.000000  1634839.0  
[2022-04-20 11:30:00, 2022-04-20 12:00:00)  92.599998  1359203.0  
[2022-04-20 13:00:00, 2022-04-20 13:30:00)  91.650002  1371977.0  
[2022-04-20 13:30:00, 2022-04-20 14:00:00)  91.150002  1052866.0  
[2022-04-20 14:00:00, 2022-04-20 14:30:00)  90.550003  1116742.0  
[2022-04-20 14:30:00, 2022-04-20 15:00:00)  90.650002  1623748.0  
[2022-04-20 15:00:00, 2022-04-20 15:30:00)  91.150002  1635377.0  
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  90.650002  2491765.0

The prices are for a single session. The Hong Kong exchange opens at 09.30 (local time) and has a morning session that runs through to 12:00. The afternoon session then opens at 13:00 and runs through to 16.00. The 30 minute interval above ensures that indices are aligned with both the am and pm closes, such that the break from 12.00 through 13.00 is manifested by the absense of indices over this period.

In both the above examples the indices align with the session\subsession closes and all indices represent only periods during which the symbols were trading. Put another way, no indice included a non-trading period.

However, when the interval is such that the indices do not align with a (sub)session close, anchoring "open" can introduce non-trading periods into the intervals.

Going back to the MSFT example, with the interval changed to 90 minutes...

In [6]:
df_90 = prices.get("90T", start_session, end_session, anchor="open")
df_90

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 09:30:00, 2022-04-19 11:00:00)  279.720001  284.750000   
[2022-04-19 11:00:00, 2022-04-19 12:30:00)  283.880005  284.820007   
[2022-04-19 12:30:00, 2022-04-19 14:00:00)  283.600006  284.059998   
[2022-04-19 14:00:00, 2022-04-19 15:30:00)  283.545013  285.582092   
[2022-04-19 15:30:00, 2022-04-19 17:00:00)  285.279999  286.170013   
[2022-04-20 09:30:00, 2022-04-20 11:00:00)  289.399994  289.700012   
[2022-04-20 11:00:00, 2022-04-20 12:30:00)  288.079987  288.130005   
[2022-04-20 12:30:00, 2022-04-20 14:00:00)  285.740997  288.010010   
[2022-04-20 14:00:00, 2022-04-20 15:30:00)  287.549988  287.980011   
[2022-04-20 15:30:00, 2022-04-20 17:00:00)  287.109985  288.119904   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 09:30:00, 2022-04-19 11:00:00)  278.410004  283.929993  6551190.0  
[2022-04-19 11:00:00, 2022-04-19 12:30:00)  283.519989  283.587799  2773877.0  
[2022-04-19 12:30:00, 2022-04-19 14:00:00)  282.980011  283.549988  2606609.0  
[2022-04-19 14:00:00, 2022-04-19 15:30:00)  282.910004  285.250000  3127720.0  
[2022-04-19 15:30:00, 2022-04-19 17:00:00)  284.500000  285.380005  3282886.0  
[2022-04-20 09:30:00, 2022-04-20 11:00:00)  285.799988  288.084991  6921787.0  
[2022-04-20 11:00:00, 2022-04-20 12:30:00)  285.370209  285.750000  3566785.0  
[2022-04-20 12:30:00, 2022-04-20 14:00:00)  285.679993  287.570007  2433980.0  
[2022-04-20 14:00:00, 2022-04-20 15:30:00)  285.519989  287.029999  3053402.0  
[2022-04-20 15:30:00, 2022-04-20 17:00:00)  286.049988  286.309998  3366341.0

The first indice again has a left side on the first session's open and indices are still contiguous through to the first session's close. However, given that the indices at this interval do not align with the 16.00 close, the last indice of the first session runs all the way through to 17.00, one hour after the close. The same is true of the last indice of the second session. These indices therefore include a one hour non-trading period following the sessions' closes.

The same effect can be seen for the Hong Kong market if the interval is changed to 40 mintues. In this case the indices no longer align with either the morning or the afternoon subsession closes.

In [7]:
df_40_hk = prices_hk.get("40T", session, session, anchor="open")
df_40_hk

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:30:00, 2022-04-20 10:10:00)  91.199997  91.900002  90.599998   
[2022-04-20 10:10:00, 2022-04-20 10:50:00)  90.900002  91.250000  90.500000   
[2022-04-20 10:50:00, 2022-04-20 11:30:00)  91.199997  92.199997  90.849998   
[2022-04-20 11:30:00, 2022-04-20 12:10:00)  92.000000  92.949997  91.949997   
[2022-04-20 13:00:00, 2022-04-20 13:40:00)  92.650002  92.750000  91.500000   
[2022-04-20 13:40:00, 2022-04-20 14:20:00)  91.699997  91.900002  90.900002   
[2022-04-20 14:20:00, 2022-04-20 15:00:00)  90.900002  91.050003  90.300003   
[2022-04-20 15:00:00, 2022-04-20 15:40:00)  90.650002  91.300003  90.550003   
[2022-04-20 15:40:00, 2022-04-20 16:20:00)  91.150002  91.300003  90.599998   

symbol                                                            
                                                close     volume  
[2022-04-20 09:30:00, 2022-04-20 10:10:00)  90.900002  2494538.0  
[2022-04-20 10:10:00, 2022-04-20 10:50:00)  91.150002  1958569.0  
[2022-04-20 10:50:00, 2022-04-20 11:30:00)  92.000000  2034541.0  
[2022-04-20 11:30:00, 2022-04-20 12:10:00)  92.599998  1359203.0  
[2022-04-20 13:00:00, 2022-04-20 13:40:00)  91.699997  1571339.0  
[2022-04-20 13:40:00, 2022-04-20 14:20:00)  90.900002  1469246.0  
[2022-04-20 14:20:00, 2022-04-20 15:00:00)  90.650002  2124748.0  
[2022-04-20 15:00:00, 2022-04-20 15:40:00)  91.150002  2500419.0  
[2022-04-20 15:40:00, 2022-04-20 16:20:00)  90.650002  1626723.0

Now the last indice of the am session runs to 12.10, 10 minutes after the 12.00 close. The following indice correctly has a left side that conincides with the pm open. The final indice can be seen to also be unaligned with the 16.00 close.

Take this away...

* **Anchoring "open" maintains the `interval` throughout**, albeit with the consequence of **including non-trading periods after (sub)session closes whenever the indices do not otherwise align with the (sub)session close.**

### Interrogating indices with `.pt` accessor

Non-trading periods can be investigated with a host of .pt price table accessor methods (the [pt_accessor tutorial](./pt_accessor.ipynb) includes further methods not covered here).

#### Trading status

`.pt.indices_all_trading` queries if all indices cover *only* trading periods *for a specific calendar*. Comparing the return for the 30\90 minute MSFT tables shows how the unaligned indices at 90 minutes introduces non-trading periods into some indices...

In [8]:
df_30.pt.indices_all_trading(xnys)

True

In [9]:
df_90.pt.indices_all_trading(xnys)

False

`.pt.indices_partial_trading` allows for querying those indices that specifically cover both trading and non-trading periods, i.e. in this case the last indices of each session...

In [10]:
df_90.pt.indices_partial_trading(xnys)

IntervalIndex([[2022-04-19 15:30:00, 2022-04-19 17:00:00), [2022-04-20 15:30:00, 2022-04-20 17:00:00)], dtype='interval[datetime64[ns, America/New_York], left]')

And for the unaligned Hong Kong 40 minute prices...

In [11]:
df_40_hk.pt.indices_partial_trading(xhkg)

IntervalIndex([[2022-04-20 11:30:00, 2022-04-20 12:10:00), [2022-04-20 15:40:00, 2022-04-20 16:20:00)], dtype='interval[datetime64[ns, Asia/Hong_Kong], left]')

For each partial trading indice `.pt.indices_partial_trading_info` details which part(s) of the indice represent non-trading periods. The return is a dictionary with keys as partial indices and values as a `pd.IntervalIndex` comprising one or more `pd.Interval` that each describe a distinct non-trading period within the indice.

In [12]:
df_90.pt.indices_partial_trading_info(xnys)

{Interval('2022-04-19 15:30:00', '2022-04-19 17:00:00', closed='left'): IntervalIndex([[2022-04-19 16:00:00, 2022-04-19 17:00:00)], dtype='interval[datetime64[ns, America/New_York], left]'),
 Interval('2022-04-20 15:30:00', '2022-04-20 17:00:00', closed='left'): IntervalIndex([[2022-04-20 16:00:00, 2022-04-20 17:00:00)], dtype='interval[datetime64[ns, America/New_York], left]')}

In [13]:
df_40_hk.pt.indices_partial_trading_info(xhkg)

{Interval('2022-04-20 11:30:00', '2022-04-20 12:10:00', closed='left'): IntervalIndex([[2022-04-20 12:00:00, 2022-04-20 12:10:00)], dtype='interval[datetime64[ns, Asia/Hong_Kong], left]'),
 Interval('2022-04-20 15:40:00', '2022-04-20 16:20:00', closed='left'): IntervalIndex([[2022-04-20 16:00:00, 2022-04-20 16:20:00)], dtype='interval[datetime64[ns, Asia/Hong_Kong], left]')}

`.pt.indices_trading` queries which indices cover only trading periods (i.e. include no non-trading periods).

In [14]:
df_90.pt.indices_trading(xnys)

IntervalIndex([[2022-04-19 09:30:00, 2022-04-19 11:00:00), [2022-04-19 11:00:00, 2022-04-19 12:30:00), [2022-04-19 12:30:00, 2022-04-19 14:00:00), [2022-04-19 14:00:00, 2022-04-19 15:30:00), [2022-04-20 09:30:00, 2022-04-20 11:00:00), [2022-04-20 11:00:00, 2022-04-20 12:30:00), [2022-04-20 12:30:00, 2022-04-20 14:00:00), [2022-04-20 14:00:00, 2022-04-20 15:30:00)], dtype='interval[datetime64[ns, America/New_York], left]')

`.pt.indices_non_trading` queries any indices that fully cover a non-trading period (i.e. no minute of the indices represent a trading minute). In this example the results are empty as all indices at least partially cover a trading period for the default calendar. (This property's use is better demonstrated later ([Multiple Calendars](#Multiple-calendars)) when the data set comprises symbols trading on different calendars.)

In [15]:
df_90.pt.indices_non_trading(xnys)

IntervalIndex([], dtype='interval[datetime64[ns, America/New_York], left]')

In [16]:
df_90.pt.indices_non_trading(xnys).empty

True

`.pt.indices_trading_status` neatly expresses the status of each indice by the value of a `pd.Series`:
* True indicates the indice fully covers a trading period.
* N/A indicates a partial trading indice.
* False indicates that indice fully covers a non-trading period.

In [17]:
df_90.pt.indices_trading_status(xnys)

[2022-04-19 09:30:00, 2022-04-19 11:00:00)    True
[2022-04-19 11:00:00, 2022-04-19 12:30:00)    True
[2022-04-19 12:30:00, 2022-04-19 14:00:00)    True
[2022-04-19 14:00:00, 2022-04-19 15:30:00)    True
[2022-04-19 15:30:00, 2022-04-19 17:00:00)     NaN
[2022-04-20 09:30:00, 2022-04-20 11:00:00)    True
[2022-04-20 11:00:00, 2022-04-20 12:30:00)    True
[2022-04-20 12:30:00, 2022-04-20 14:00:00)    True
[2022-04-20 14:00:00, 2022-04-20 15:30:00)    True
[2022-04-20 15:30:00, 2022-04-20 17:00:00)     NaN
dtype: object

In [18]:
df_40_hk.pt.indices_trading_status(xhkg)

[2022-04-20 09:30:00, 2022-04-20 10:10:00)    True
[2022-04-20 10:10:00, 2022-04-20 10:50:00)    True
[2022-04-20 10:50:00, 2022-04-20 11:30:00)    True
[2022-04-20 11:30:00, 2022-04-20 12:10:00)     NaN
[2022-04-20 13:00:00, 2022-04-20 13:40:00)    True
[2022-04-20 13:40:00, 2022-04-20 14:20:00)    True
[2022-04-20 14:20:00, 2022-04-20 15:00:00)    True
[2022-04-20 15:00:00, 2022-04-20 15:40:00)    True
[2022-04-20 15:40:00, 2022-04-20 16:20:00)     NaN
dtype: object

#### Number of trading minutes
Various `.pt` accessor methods consider the trading minutes contained by each interval.

(In addition to the methods shown here, `pt.trading_minutes_interval` is demonstrated in the [anchor "workback"](#anchor-"workback") section.)

`.pt.indices_trading_minutes` returns a `pd.Series` that shows the number of trading minutes associated with each indice.

In [19]:
df_90.pt.indices_trading_minutes(xnys)

[2022-04-19 09:30:00, 2022-04-19 11:00:00)    90
[2022-04-19 11:00:00, 2022-04-19 12:30:00)    90
[2022-04-19 12:30:00, 2022-04-19 14:00:00)    90
[2022-04-19 14:00:00, 2022-04-19 15:30:00)    90
[2022-04-19 15:30:00, 2022-04-19 17:00:00)    30
[2022-04-20 09:30:00, 2022-04-20 11:00:00)    90
[2022-04-20 11:00:00, 2022-04-20 12:30:00)    90
[2022-04-20 12:30:00, 2022-04-20 14:00:00)    90
[2022-04-20 14:00:00, 2022-04-20 15:30:00)    90
[2022-04-20 15:30:00, 2022-04-20 17:00:00)    30
Name: trading_mins, dtype: int64

In [20]:
df_40_hk.pt.indices_trading_minutes(xhkg)

[2022-04-20 09:30:00, 2022-04-20 10:10:00)    40
[2022-04-20 10:10:00, 2022-04-20 10:50:00)    40
[2022-04-20 10:50:00, 2022-04-20 11:30:00)    40
[2022-04-20 11:30:00, 2022-04-20 12:10:00)    30
[2022-04-20 13:00:00, 2022-04-20 13:40:00)    40
[2022-04-20 13:40:00, 2022-04-20 14:20:00)    40
[2022-04-20 14:20:00, 2022-04-20 15:00:00)    40
[2022-04-20 15:00:00, 2022-04-20 15:40:00)    40
[2022-04-20 15:40:00, 2022-04-20 16:20:00)    20
Name: trading_mins, dtype: int64

`.pt.indices_have_trading_minutes` queries if all indices have the same number of trading minutes.

In [21]:
df_90.pt.indices_have_regular_trading_minutes(xnys)

False

...although that for the 30 minute interval table

In [22]:
df_30.pt.indices_have_regular_trading_minutes(xnys)

True

#### Intervals' lengths
Some .pt accessor methods provide for querying the intervals' lengths (in terms of absolute minutes, rather than trading minutes).

(In addition to the methods shown here, `pt.by_indice_length` is demonstrated in the [Indices overlapping](#Indices-overlapping-next-session) section.)

`pt.has_regular_interval` queries if all indices have the same length.

In [23]:
df_90.pt.has_regular_interval

True

Lastly, `.pt.indices_length` returns a pd.Series indexed with all interval lengths and with values indicating the number of indices of each length. When prices are anchored "open" all indices will have the same length (unless [warned](#Indices-overlapping-next-session) otherwise) ...

In [24]:
df_90.pt.indices_length

0 days 01:30:00    10
dtype: int64

unless indices are forced...

### `force`

The `force` option can be passed to **curtail indices that include a single non-trading period so that they instead cover only the period during which at least one symbol was trading**.

The effect can be seen by repeating the MSFT 90 minute example with `force` as True.

In [25]:
df_90_f = prices.get(
    "90T", start_session, end_session, anchor="open", force=True
)
df_90_f

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 09:30:00, 2022-04-19 11:00:00)  279.720001  284.750000   
[2022-04-19 11:00:00, 2022-04-19 12:30:00)  283.880005  284.820007   
[2022-04-19 12:30:00, 2022-04-19 14:00:00)  283.600006  284.059998   
[2022-04-19 14:00:00, 2022-04-19 15:30:00)  283.545013  285.582092   
[2022-04-19 15:30:00, 2022-04-19 16:00:00)  285.279999  286.170013   
[2022-04-20 09:30:00, 2022-04-20 11:00:00)  289.399994  289.700012   
[2022-04-20 11:00:00, 2022-04-20 12:30:00)  288.079987  288.130005   
[2022-04-20 12:30:00, 2022-04-20 14:00:00)  285.740997  288.010010   
[2022-04-20 14:00:00, 2022-04-20 15:30:00)  287.549988  287.980011   
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  287.109985  288.119904   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 09:30:00, 2022-04-19 11:00:00)  278.410004  283.929993  6551190.0  
[2022-04-19 11:00:00, 2022-04-19 12:30:00)  283.519989  283.587799  2773877.0  
[2022-04-19 12:30:00, 2022-04-19 14:00:00)  282.980011  283.549988  2606609.0  
[2022-04-19 14:00:00, 2022-04-19 15:30:00)  282.910004  285.250000  3127720.0  
[2022-04-19 15:30:00, 2022-04-19 16:00:00)  284.500000  285.380005  3282886.0  
[2022-04-20 09:30:00, 2022-04-20 11:00:00)  285.799988  288.084991  6921787.0  
[2022-04-20 11:00:00, 2022-04-20 12:30:00)  285.370209  285.750000  3566785.0  
[2022-04-20 12:30:00, 2022-04-20 14:00:00)  285.679993  287.570007  2433980.0  
[2022-04-20 14:00:00, 2022-04-20 15:30:00)  285.519989  287.029999  3053402.0  
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  286.049988  286.309998  3366341.0

The indices at the end of each session can now be seen to reflect the 16.00 close. A consequence of forcing is that the indices are no longer all of the same length...

In [26]:
df_90_f.pt.has_regular_interval

False

In [27]:
df_90_f.pt.indices_length

0 days 01:30:00    8
0 days 00:30:00    2
dtype: int64

Although now they all only represent trading periods, i.e. there are no partial trading indices...

In [28]:
df_90_f.pt.indices_all_trading(xnys)

True

In [29]:
df_90_f.pt.indices_partial_trading(xnys).empty

True

Forcing the previous 40 minute Hong Kong example shows how force will force the last indices of both the am and pm subsessions to their respective closes.

In [30]:
df_40_hk_f = prices_hk.get(
    "40T", session, session, anchor="open", force=True
)
df_40_hk_f

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:30:00, 2022-04-20 10:10:00)  91.199997  91.900002  90.599998   
[2022-04-20 10:10:00, 2022-04-20 10:50:00)  90.900002  91.250000  90.500000   
[2022-04-20 10:50:00, 2022-04-20 11:30:00)  91.199997  92.199997  90.849998   
[2022-04-20 11:30:00, 2022-04-20 12:00:00)  92.000000  92.949997  91.949997   
[2022-04-20 13:00:00, 2022-04-20 13:40:00)  92.650002  92.750000  91.500000   
[2022-04-20 13:40:00, 2022-04-20 14:20:00)  91.699997  91.900002  90.900002   
[2022-04-20 14:20:00, 2022-04-20 15:00:00)  90.900002  91.050003  90.300003   
[2022-04-20 15:00:00, 2022-04-20 15:40:00)  90.650002  91.300003  90.550003   
[2022-04-20 15:40:00, 2022-04-20 16:00:00)  91.150002  91.300003  90.599998   

symbol                                                            
                                                close     volume  
[2022-04-20 09:30:00, 2022-04-20 10:10:00)  90.900002  2494538.0  
[2022-04-20 10:10:00, 2022-04-20 10:50:00)  91.150002  1958569.0  
[2022-04-20 10:50:00, 2022-04-20 11:30:00)  92.000000  2034541.0  
[2022-04-20 11:30:00, 2022-04-20 12:00:00)  92.599998  1359203.0  
[2022-04-20 13:00:00, 2022-04-20 13:40:00)  91.699997  1571339.0  
[2022-04-20 13:40:00, 2022-04-20 14:20:00)  90.900002  1469246.0  
[2022-04-20 14:20:00, 2022-04-20 15:00:00)  90.650002  2124748.0  
[2022-04-20 15:00:00, 2022-04-20 15:40:00)  91.150002  2500419.0  
[2022-04-20 15:40:00, 2022-04-20 16:00:00)  90.650002  1626723.0

In [31]:
df_40_hk_f.pt.indices_length

0 days 00:40:00    7
0 days 00:30:00    1
0 days 00:20:00    1
dtype: int64

Worth noting that the force option only effects the index, not the prices of the forced indices (as no symbol trades during the non-trading period being curtailed, the price data is unchanged).

#### Force to period end with `openend`

The Golden Rule (see [periods tutorial](./periods.ipynb)) dictates that prices should never be included for any period that lies before `start` or after `end`. However, the final *indice* can extend beyond `end` so long as this does not result in the incorporation of prices registered after `end`.

Consider the following example which requests prices between the open and close of a single session.

In [32]:
start, end = xnys.session_open_close(session)
# put in terms of local tz for ease of reference
start = start.astimezone(prices.tz_default)
end = end.astimezone(prices.tz_default)
# for reference
start, end

(Timestamp('2022-04-20 09:30:00-0400', tz='America/New_York'),
 Timestamp('2022-04-20 16:00:00-0400', tz='America/New_York'))

In [33]:
df_maintain = prices.get("1H", start, end)
df_maintain

symbol                                            MSFT              \
                                                  open        high   
[2022-04-20 09:30:00, 2022-04-20 10:30:00)  289.399994  289.700012   
[2022-04-20 10:30:00, 2022-04-20 11:30:00)  288.149994  288.890015   
[2022-04-20 11:30:00, 2022-04-20 12:30:00)  286.519989  287.600006   
[2022-04-20 12:30:00, 2022-04-20 13:30:00)  285.740997  287.369995   
[2022-04-20 13:30:00, 2022-04-20 14:30:00)  286.880005  288.010010   
[2022-04-20 14:30:00, 2022-04-20 15:30:00)  286.959991  287.399994   
[2022-04-20 15:30:00, 2022-04-20 16:30:00)  287.109985  288.119904   

symbol                                                                         
                                                   low       close     volume  
[2022-04-20 09:30:00, 2022-04-20 10:30:00)  285.799988  288.100006  5723475.0  
[2022-04-20 10:30:00, 2022-04-20 11:30:00)  286.220001  286.510010  2621559.0  
[2022-04-20 11:30:00, 2022-04-20 12:30:00)  285.370209  285.750000  2143538.0  
[2022-04-20 12:30:00, 2022-04-20 13:30:00)  285.679993  286.929993  1644318.0  
[2022-04-20 13:30:00, 2022-04-20 14:30:00)  286.750000  286.950012  1613055.0  
[2022-04-20 14:30:00, 2022-04-20 15:30:00)  285.519989  287.029999  2230009.0  
[2022-04-20 15:30:00, 2022-04-20 16:30:00)  286.049988  286.309998  3366341.0

Although `end` was 16:00, the last indice runs to 16:30. The one hour interval results in the period end, 16:30, being unaligned with the indices. Such a period end is considered to be an **unaligned session close**. The `openend` option exists to choose **how to define the final indice when the period end is an unaligned session close**.

The default `openend` is **"maintain"**. This maintains the `interval` by ensuring that the last indice has the same length as all the rest. This is what happened above.

In [34]:
df_maintain.pt.has_regular_interval

True

However, the only reason that the final indice above was allowed to extend beyond `end` is that no symbol traded during the period between the unaligned session close and the end of the indice that contained it. If a symbol does trade during this period then the table ends with the indice that preceeds the unaligned session close (examples of this behaviour are included to the [openend](#openend) section of [multiple calendars](#Multiple-calendars)).

If representing the period end is more important than maintaining the interval then `openend` can be passed as **shorten** to shorten the final indice to represent only the period through to the session close.

In [35]:
df_shorten = prices.get("1H", start, end, openend="shorten")
df_shorten

symbol                                            MSFT              \
                                                  open        high   
[2022-04-20 09:30:00, 2022-04-20 10:30:00)  289.399994  289.700012   
[2022-04-20 10:30:00, 2022-04-20 11:30:00)  288.149994  288.890015   
[2022-04-20 11:30:00, 2022-04-20 12:30:00)  286.519989  287.600006   
[2022-04-20 12:30:00, 2022-04-20 13:30:00)  285.740997  287.369995   
[2022-04-20 13:30:00, 2022-04-20 14:30:00)  286.880005  288.010010   
[2022-04-20 14:30:00, 2022-04-20 15:30:00)  286.959991  287.399994   
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  287.109985  288.119904   

symbol                                                                         
                                                   low       close     volume  
[2022-04-20 09:30:00, 2022-04-20 10:30:00)  285.799988  288.100006  5723475.0  
[2022-04-20 10:30:00, 2022-04-20 11:30:00)  286.220001  286.510010  2621559.0  
[2022-04-20 11:30:00, 2022-04-20 12:30:00)  285.370209  285.750000  2143538.0  
[2022-04-20 12:30:00, 2022-04-20 13:30:00)  285.679993  286.929993  1644318.0  
[2022-04-20 13:30:00, 2022-04-20 14:30:00)  286.750000  286.950012  1613055.0  
[2022-04-20 14:30:00, 2022-04-20 15:30:00)  285.519989  287.029999  2230009.0  
[2022-04-20 15:30:00, 2022-04-20 16:00:00)  286.049988  286.309998  3366341.0

Notice that the final indice now ends at 16:00 and hence is shorter than `interval`, covering only 30 minutes.

In [36]:
df_shorten.pt.has_regular_interval

False

Note that it's only possible for the period end to fall on an unaligned session close if the `anchor` is "open" (when [`anchor` is "workback"](#anchor-"workback") the period end is by definition aligned with the right of the last indice). Hence the name of the `openend` option - choose what to do with the end when the anchor is "open".

The `openend` option is further explored in the [openend](#openend) section of [multiple calendars](#Multiple-calendars), including an example of when it's not possible to 'shorten' the final indice as a result of data availability.

### Indices overlapping next session

If the `interval` is long relative to the gap between a session close and the next session open then it's possible for the last indice of a session to overlap the first indice of the next session. This is easily illustrated with a symbol that trades 24/7 and therefore has no gaps between consecutive sessions. Bitcoin for example.

With an 8 hour interval the indices coincide with the sessions' closes and the intervals are all regular.

In [37]:
prices_btc = PricesYahoo("BTC-USD")
print(f"{start_session=}\n{end_session=}\n")  # for reference

df = prices_btc.get("8H", start_session, end_session, anchor="open")
df

start_session=Timestamp('2022-04-19 00:00:00')
end_session=Timestamp('2022-04-20 00:00:00')



symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-19, 2022-04-19 08:00:00)           40828.175781  41146.886719   
[2022-04-19 08:00:00, 2022-04-19 16:00:00)  40685.148438  41672.960938   
[2022-04-19 16:00:00, 2022-04-20)           41467.316406  41525.679688   
[2022-04-20, 2022-04-20 08:00:00)           41495.515625  41526.152344   
[2022-04-20 08:00:00, 2022-04-20 16:00:00)  41355.664062  42126.300781   
[2022-04-20 16:00:00, 2022-04-21)           41267.566406  41571.660156   

symbol                                                                  \
                                                     low         close   
[2022-04-19, 2022-04-19 08:00:00)           40618.632812  40674.910156   
[2022-04-19 08:00:00, 2022-04-19 16:00:00)  40636.628906  41486.648438   
[2022-04-19 16:00:00, 2022-04-20)           41239.843750  41502.281250   
[2022-04-20, 2022-04-20 08:00:00)           41278.414062  41353.929688   
[2022-04-20 08:00:00, 2022-04-20 16:00:00)  41141.402344  41273.136719   
[2022-04-20 16:00:00, 2022-04-21)           40961.097656  41377.843750   

symbol                                                    
                                                  volume  
[2022-04-19, 2022-04-19 08:00:00)           9.237709e+08  
[2022-04-19 08:00:00, 2022-04-19 16:00:00)  2.339473e+09  
[2022-04-19 16:00:00, 2022-04-20)           0.000000e+00  
[2022-04-20, 2022-04-20 08:00:00)           7.351460e+08  
[2022-04-20 08:00:00, 2022-04-20 16:00:00)  3.405101e+09  
[2022-04-20 16:00:00, 2022-04-21)           1.670937e+09

In [38]:
df.pt.has_regular_interval

True

As bitcoin trades 24/7 and the interval is 8 hours, there are 3 indices for each of the two sessions (bitcoin is associated with the 24/7 calendar which has opens and closes as UTC midnight).

Look at what happens if the interval is changed to 7 hours, which doesn't align with the close.

In [ ]:
df_ol = prices_btc.get("7H", start_session, end_session, anchor="open")

```
IntervalIrregularWarning: 
PriceTable interval is irregular. One or more indices were curtailed to prevent the last indice assigned to a (sub)session from overlapping with the first indice of the following (sub)session.
Use .pt.indices_length and .pt.by_indice_length to interrogate.
  warnings.warn(errors.IntervalIrregularWarning())
```

In [40]:
df_ol

symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-19, 2022-04-19 07:00:00)           40828.175781  41146.886719   
[2022-04-19 07:00:00, 2022-04-19 14:00:00)  40719.832031  41392.675781   
[2022-04-19 14:00:00, 2022-04-19 21:00:00)  41432.988281  41672.960938   
[2022-04-19 21:00:00, 2022-04-20)           41300.472656  41510.414062   
[2022-04-20, 2022-04-20 07:00:00)           41495.515625  41514.226562   
[2022-04-20 07:00:00, 2022-04-20 14:00:00)  41444.296875  42126.300781   
[2022-04-20 14:00:00, 2022-04-20 21:00:00)  41594.031250  41711.167969   

symbol                                                                  \
                                                     low         close   
[2022-04-19, 2022-04-19 07:00:00)           40648.042969  40732.976562   
[2022-04-19 07:00:00, 2022-04-19 14:00:00)  40618.632812  41392.675781   
[2022-04-19 14:00:00, 2022-04-19 21:00:00)  41239.843750  41293.839844   
[2022-04-19 21:00:00, 2022-04-20)           41274.035156  41502.281250   
[2022-04-20, 2022-04-20 07:00:00)           41278.414062  41441.253906   
[2022-04-20 07:00:00, 2022-04-20 14:00:00)  41332.937500  41612.906250   
[2022-04-20 14:00:00, 2022-04-20 21:00:00)  40961.097656  41405.281250   

symbol                                                    
                                                  volume  
[2022-04-19, 2022-04-19 07:00:00)           9.237709e+08  
[2022-04-19 07:00:00, 2022-04-19 14:00:00)  1.097351e+09  
[2022-04-19 14:00:00, 2022-04-19 21:00:00)  1.242122e+09  
[2022-04-19 21:00:00, 2022-04-20)           0.000000e+00  
[2022-04-20, 2022-04-20 07:00:00)           4.293530e+08  
[2022-04-20 07:00:00, 2022-04-20 14:00:00)  3.207594e+09  
[2022-04-20 14:00:00, 2022-04-20 21:00:00)  1.653217e+09

An `IntervalIrregularWarning` was raised advising that one or more indices had to be curtailed to prevent them overlapping with the next indices. This is raised to avoid the user implicitly expecting all intervals to be regular.

As suggested by the warning, `.pt` accessor methods can be used to interrogate what's going on.

`pt.has_regular_interval` indicates that the interval is indeed irregular... 

In [41]:
df_ol.pt.has_regular_interval

False

`pt.indices_length` shows that whilst most indices are the requested 7 hours, one has been curtailed to 3 hours.

In [42]:
df_ol.pt.indices_length

0 days 07:00:00    6
0 days 03:00:00    1
dtype: int64

We can also refer to `pt.by_indice_length` which returns a geneartor that iterates through 2-tuples that each offer an interval length and a `pd.DataFrame` comprised of all the indices of that interval length.

In [43]:
gen = df_ol.pt.by_indice_length
interval, df = next(gen)
print(interval)
df

0 days 03:00:00


symbol                                  BTC-USD                              \
                                           open          high           low   
[2022-04-19 21:00:00, 2022-04-20)  41300.472656  41510.414062  41274.035156   

symbol                                                 
                                         close volume  
[2022-04-19 21:00:00, 2022-04-20)  41502.28125    0.0

The first iteration shows us the indice with 3 hour duration, i.e. the indice at the end of the first session which had to be curtailed in order to not overlap the first indice of the second session.

The next iteration gives all the indices with the 7 hour intervals.

In [44]:
interval, df = next(gen)
print(interval)
df

0 days 07:00:00


symbol                                           BTC-USD                \
                                                    open          high   
[2022-04-19, 2022-04-19 07:00:00)           40828.175781  41146.886719   
[2022-04-19 07:00:00, 2022-04-19 14:00:00)  40719.832031  41392.675781   
[2022-04-19 14:00:00, 2022-04-19 21:00:00)  41432.988281  41672.960938   
[2022-04-20, 2022-04-20 07:00:00)           41495.515625  41514.226562   
[2022-04-20 07:00:00, 2022-04-20 14:00:00)  41444.296875  42126.300781   
[2022-04-20 14:00:00, 2022-04-20 21:00:00)  41594.031250  41711.167969   

symbol                                                                  \
                                                     low         close   
[2022-04-19, 2022-04-19 07:00:00)           40648.042969  40732.976562   
[2022-04-19 07:00:00, 2022-04-19 14:00:00)  40618.632812  41392.675781   
[2022-04-19 14:00:00, 2022-04-19 21:00:00)  41239.843750  41293.839844   
[2022-04-20, 2022-04-20 07:00:00)           41278.414062  41441.253906   
[2022-04-20 07:00:00, 2022-04-20 14:00:00)  41332.937500  41612.906250   
[2022-04-20 14:00:00, 2022-04-20 21:00:00)  40961.097656  41405.281250   

symbol                                                    
                                                  volume  
[2022-04-19, 2022-04-19 07:00:00)           9.237709e+08  
[2022-04-19 07:00:00, 2022-04-19 14:00:00)  1.097351e+09  
[2022-04-19 14:00:00, 2022-04-19 21:00:00)  1.242122e+09  
[2022-04-20, 2022-04-20 07:00:00)           4.293530e+08  
[2022-04-20 07:00:00, 2022-04-20 14:00:00)  3.207594e+09  
[2022-04-20 14:00:00, 2022-04-20 21:00:00)  1.653217e+09

Overlapping indices can also be a common issue when symbols have a break. Going back to the prices of Alibaba, look at what happens when the interval is judiciously set to 106 minutes...

In [ ]:
df_106 = prices_hk.get("106T", session, session, anchor="open")

```
IntervalIrregularWarning: 
PriceTable interval is irregular. One or more indices were curtailed to prevent the last indice assigned to a (sub)session from overlapping with the first indice of the following (sub)session.
Use .pt.indices_length and .pt.by_indice_length to interrogate.
  warnings.warn(errors.IntervalIrregularWarning())
```

In [46]:
df_106

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:30:00, 2022-04-20 11:16:00)  91.199997  92.050003  90.500000   
[2022-04-20 11:16:00, 2022-04-20 13:00:00)  92.000000  92.949997  91.800003   
[2022-04-20 13:00:00, 2022-04-20 14:46:00)  92.650002  92.750000  90.300003   
[2022-04-20 14:46:00, 2022-04-20 16:32:00)  90.849998  91.300003  90.550003   

symbol                                                            
                                                close     volume  
[2022-04-20 09:30:00, 2022-04-20 11:16:00)  92.050003  6132298.0  
[2022-04-20 11:16:00, 2022-04-20 13:00:00)  92.599998  2269680.0  
[2022-04-20 13:00:00, 2022-04-20 14:46:00)  90.800003  4587665.0  
[2022-04-20 14:46:00, 2022-04-20 16:32:00)  90.650002  5443951.0

106 minutes is just long enough for the last indice of the am session (closing 12:00) to overlap (by one minute) with the first indice of the pm session (opening 13:00).

For what's its worth, in this case all intervals will be the same length if the interval is increased to a value that will reduce the am session to a single indice...

In [47]:
prices_hk.get("150T", session, session, anchor="open")

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:30:00, 2022-04-20 12:00:00)  91.199997  92.949997  90.500000   
[2022-04-20 13:00:00, 2022-04-20 15:30:00)  92.650002  92.750000  90.300003   
[2022-04-20 15:30:00, 2022-04-20 18:00:00)  91.099998  91.300003  90.599998   

symbol                                                            
                                                close     volume  
[2022-04-20 09:30:00, 2022-04-20 12:00:00)  92.599998  7846851.0  
[2022-04-20 13:00:00, 2022-04-20 15:30:00)  91.150002  6800710.0  
[2022-04-20 15:30:00, 2022-04-20 18:00:00)  90.650002  2491765.0

## `anchor` "workback"

Whilst prices are traditionally anchored on the 'open', `market_prices` also offers the "workback" option to **anchor the indices at the end of the evaluated period**. The indices are then evaluated by 'working back' the number of **trading minutes** represented by the `interval`.

Using the MSFT prices, consider how indices at 90 minute intervals are evaluated when working back from the second session's close.

In [48]:
df_wb_90 = prices.get("90T", start_session, end_session, anchor="workback")
df_wb_90

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 10:30:00, 2022-04-19 12:00:00)  283.359985  284.820007   
[2022-04-19 12:00:00, 2022-04-19 13:30:00)  284.269989  284.269989   
[2022-04-19 13:30:00, 2022-04-19 15:00:00)  283.179993  284.359985   
[2022-04-19 15:00:00, 2022-04-20 10:00:00)  284.279999  289.700012   
[2022-04-20 10:00:00, 2022-04-20 11:30:00)  286.380005  288.890015   
[2022-04-20 11:30:00, 2022-04-20 13:00:00)  286.519989  287.600006   
[2022-04-20 13:00:00, 2022-04-20 14:30:00)  286.459991  288.010010   
[2022-04-20 14:30:00, 2022-04-20 16:00:00)  286.959991  288.119904   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 10:30:00, 2022-04-19 12:00:00)  283.230011  284.359985  3348685.0  
[2022-04-19 12:00:00, 2022-04-19 13:30:00)  283.149994  283.200012  2297989.0  
[2022-04-19 13:30:00, 2022-04-19 15:00:00)  282.910004  284.260010  2854030.0  
[2022-04-19 15:00:00, 2022-04-20 10:00:00)  284.049988  286.399994  8375341.0  
[2022-04-20 10:00:00, 2022-04-20 11:30:00)  285.799988  286.510010  4534175.0  
[2022-04-20 11:30:00, 2022-04-20 13:00:00)  285.370209  286.480011  2977636.0  
[2022-04-20 13:00:00, 2022-04-20 14:30:00)  286.220001  286.950012  2423275.0  
[2022-04-20 14:30:00, 2022-04-20 16:00:00)  285.519989  286.309998  5596350.0

Whereas when prices were anchored "open" the indices were unaligned with the last session close, now the right of the last indice coincides with the session end. Each preceeding indice represents the 90 trading minutes that preceed the following indice.

The consequence of working back in terms of trading minutes is most salient where this results in indices 'crossing sessions', as occurs above in the 4th indice.

In [49]:
df_wb_90.iloc[[3]]

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 15:00:00, 2022-04-20 10:00:00)  284.279999  289.700012   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 15:00:00, 2022-04-20 10:00:00)  284.049988  286.399994  8375341.0

This indice runs from 15:00 in the first session through to 10:00 in the second session. This is a huge interval in absolute terms, some 19 hours, when the interval was for only 90 minutes.

In [50]:
df_wb_90.pt.indices_length

0 days 01:30:00    7
0 days 19:00:00    1
dtype: int64

It's a massive partial trading indice that includes the full non-trading period between the first session's close and the second session's open.

In [51]:
df_wb_90.pt.indices_partial_trading_info(xnys)

{Interval('2022-04-19 15:00:00', '2022-04-20 10:00:00', closed='left'): IntervalIndex([[2022-04-19 16:00:00, 2022-04-20 09:29:00)], dtype='interval[datetime64[ns, America/New_York], left]')}

HOWEVER, this indice covers 90 trading minutes in the same way that ALL the other indices cover 90 trading minutes. In this case the 90 minutes comprise an hour from the first session, 15:00 through to the 16:00 close, and half an hour from the second session, from the 09:30 open through to 10:00.

The .pt accessor methods that query trading minutes show this uniformity...

In [52]:
df_wb_90.pt.indices_have_regular_trading_minutes(xnys)

True

If the table represents a constant number of trading minutes then the `.pt.trading_minutes_interval` will return that interval as a `TDInterval`.

In [53]:
df_wb_90.pt.trading_minutes_interval(xnys)

<TDInterval.T90: Timedelta('0 days 01:30:00')>

`.pt.indices_trading_minutes` shows that all the indices do indeed represent the same number of trading minutes.

In [54]:
df_wb_90.pt.indices_trading_minutes(xnys)

[2022-04-19 10:30:00, 2022-04-19 12:00:00)    90
[2022-04-19 12:00:00, 2022-04-19 13:30:00)    90
[2022-04-19 13:30:00, 2022-04-19 15:00:00)    90
[2022-04-19 15:00:00, 2022-04-20 10:00:00)    90
[2022-04-20 10:00:00, 2022-04-20 11:30:00)    90
[2022-04-20 11:30:00, 2022-04-20 13:00:00)    90
[2022-04-20 13:00:00, 2022-04-20 14:30:00)    90
[2022-04-20 14:30:00, 2022-04-20 16:00:00)    90
Name: trading_mins, dtype: int64

When a session has a break, anchoring "workback" can result in indices encompassing breaks in the same way that the indice above encompasses the gap between consecutive sessions.

In [55]:
df_wb_40_hk = prices_hk.get("40T", session, session, anchor="workback")
df_wb_40_hk

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:40:00, 2022-04-20 10:20:00)  91.400002  91.900002  90.750000   
[2022-04-20 10:20:00, 2022-04-20 11:00:00)  90.949997  91.250000  90.500000   
[2022-04-20 11:00:00, 2022-04-20 11:40:00)  91.099998  92.699997  91.050003   
[2022-04-20 11:40:00, 2022-04-20 13:20:00)  92.500000  92.949997  91.650002   
[2022-04-20 13:20:00, 2022-04-20 14:00:00)  91.650002  91.900002  91.050003   
[2022-04-20 14:00:00, 2022-04-20 14:40:00)  91.150002  91.349998  90.300003   
[2022-04-20 14:40:00, 2022-04-20 15:20:00)  90.599998  91.050003  90.550003   
[2022-04-20 15:20:00, 2022-04-20 16:00:00)  90.900002  91.300003  90.599998   

symbol                                                            
                                                close     volume  
[2022-04-20 09:40:00, 2022-04-20 10:20:00)  90.949997  2292926.0  
[2022-04-20 10:20:00, 2022-04-20 11:00:00)  91.099998  1875731.0  
[2022-04-20 11:00:00, 2022-04-20 11:40:00)  92.449997  2104439.0  
[2022-04-20 11:40:00, 2022-04-20 13:20:00)  91.699997  1815480.0  
[2022-04-20 13:20:00, 2022-04-20 14:00:00)  91.150002  1498966.0  
[2022-04-20 14:00:00, 2022-04-20 14:40:00)  90.650002  1826016.0  
[2022-04-20 14:40:00, 2022-04-20 15:20:00)  90.900002  1881642.0  
[2022-04-20 15:20:00, 2022-04-20 16:00:00)  90.650002  3159974.0

`.pt.indices_partial_trading_info` can be used here to identify the interval that encompassed the break.

In [56]:
df_wb_40_hk.pt.indices_partial_trading_info(xhkg)

{Interval('2022-04-20 11:40:00', '2022-04-20 13:20:00', closed='left'): IntervalIndex([[2022-04-20 12:00:00, 2022-04-20 12:59:00)], dtype='interval[datetime64[ns, Asia/Hong_Kong], left]')}

Also note that the first indice above does not cover the first session open (09:30). Doing so would require:
* either the first indice to comprise only 10 minute of data, i.e. fewer trading minutes than the requested `interval`.
* or the first indice to have a left side prior to requested `start` (in this case the session start). In accordance with the Golder Rule, price data is never included for periods outside of `start` or `end`.

"workback" is uesful for **ensuring that prices end on a specific timestamp**.

In [57]:
# get a specific timestamp
end = xnys.session_close(start_session) - pd.Timedelta(43, "T")
end.tz_convert(prices.tz_default)

Timestamp('2022-04-19 15:17:00-0400', tz='America/New_York')

In [58]:
prices.get("30T", end=end, hours=4, anchor="workback")

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 11:17:00, 2022-04-19 11:47:00)  283.980011  284.600006   
[2022-04-19 11:47:00, 2022-04-19 12:17:00)  284.440002  284.820007   
[2022-04-19 12:17:00, 2022-04-19 12:47:00)  283.910004  283.964996   
[2022-04-19 12:47:00, 2022-04-19 13:17:00)  283.609985  284.059998   
[2022-04-19 13:17:00, 2022-04-19 13:47:00)  283.667603  283.920013   
[2022-04-19 13:47:00, 2022-04-19 14:17:00)  283.200012  284.019989   
[2022-04-19 14:17:00, 2022-04-19 14:47:00)  283.299988  283.859985   
[2022-04-19 14:47:00, 2022-04-19 15:17:00)  283.790009  285.369995   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 11:17:00, 2022-04-19 11:47:00)  283.600006  284.429993  1020456.0  
[2022-04-19 11:47:00, 2022-04-19 12:17:00)  283.690002  283.924988   760348.0  
[2022-04-19 12:17:00, 2022-04-19 12:47:00)  283.290009  283.619995   728610.0  
[2022-04-19 12:47:00, 2022-04-19 13:17:00)  283.320007  283.640106   782565.0  
[2022-04-19 13:17:00, 2022-04-19 13:47:00)  282.980011  283.204010   875449.0  
[2022-04-19 13:47:00, 2022-04-19 14:17:00)  282.970001  283.290009  1150876.0  
[2022-04-19 14:17:00, 2022-04-19 14:47:00)  282.910004  283.774994   874043.0  
[2022-04-19 14:47:00, 2022-04-19 15:17:00)  283.670013  285.350006  1046369.0

Compare the above with the output if anchor is "open"

In [59]:
df = prices.get("30T", end=end, hours=4, anchor="open")
df

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 11:00:00, 2022-04-19 11:30:00)  283.880005  284.662201   
[2022-04-19 11:30:00, 2022-04-19 12:00:00)  284.410004  284.820007   
[2022-04-19 12:00:00, 2022-04-19 12:30:00)  284.269989  284.269989   
[2022-04-19 12:30:00, 2022-04-19 13:00:00)  283.600006  283.989990   
[2022-04-19 13:00:00, 2022-04-19 13:30:00)  283.780304  284.059998   
[2022-04-19 13:30:00, 2022-04-19 14:00:00)  283.179993  284.019989   
[2022-04-19 14:00:00, 2022-04-19 14:30:00)  283.545013  283.839996   
[2022-04-19 14:30:00, 2022-04-19 15:00:00)  283.519989  284.359985   

symbol                                                                         
                                                   low       close     volume  
[2022-04-19 11:00:00, 2022-04-19 11:30:00)  283.600006  284.420013  1236148.0  
[2022-04-19 11:30:00, 2022-04-19 12:00:00)  283.899994  284.359985   838443.0  
[2022-04-19 12:00:00, 2022-04-19 12:30:00)  283.519989  283.587799   699286.0  
[2022-04-19 12:30:00, 2022-04-19 13:00:00)  283.290009  283.809998   810317.0  
[2022-04-19 13:00:00, 2022-04-19 13:30:00)  283.149994  283.200012   788386.0  
[2022-04-19 13:30:00, 2022-04-19 14:00:00)  282.980011  283.549988  1007906.0  
[2022-04-19 14:00:00, 2022-04-19 14:30:00)  282.910004  283.519989   967876.0  
[2022-04-19 14:30:00, 2022-04-19 15:00:00)  283.179993  284.260010   878248.0

When anchor is "open" the last indice falls short of the required `end` as the indices are restricted to being anchored off the session open. In contrast "workback" allows for data to be included all the way up to `end`, making the price at `end` available...

In [60]:
df.iloc[-1][("MSFT", "close")]

284.260009765625

Last point, `force` cannot be passed when anchor is "workback" - which would defeat the object of having each interval representing the same number of trading minutes.

Take this away...

When the anchor is "workback":
* the right of the last indice will align with the period end.
* **all indices will cover the same number of trading minutes** (as `interval`).
* **non-tading periods between (sub)sessions will be included within indices** whenever the indices do not otherwise align with the sub(session) open.

So, is it better to anchor prices on the "open" or "workback"? Depends what you're trying to do. `market_prices` just tries to make useful options available.

## Multiple calendars

### `anchor` "open" with multiple calendars

When symbols are associated with multiple calendars (see [prices](./prices.ipynb) tutorial) prices anchored "open" will be anchored to the sessions' opens of the calendar associated with the lead symbol. Consider the following prices for Microsoft, listed in New York, and AstraZeneca, listed in London.

In [61]:
prices_us_lon = PricesYahoo("MSFT AZN.L")
print(f"{start_session=}\n{end_session=}\n")  # for reference

prices_us_lon.get("1H", start_session, end_session, lead_symbol="AZN.L")

start_session=Timestamp('2022-04-19 00:00:00')
end_session=Timestamp('2022-04-20 00:00:00')



symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 08:00:00, 2022-04-19 09:00:00)         NaN         NaN   
[2022-04-19 09:00:00, 2022-04-19 10:00:00)         NaN         NaN   
[2022-04-19 10:00:00, 2022-04-19 11:00:00)         NaN         NaN   
[2022-04-19 11:00:00, 2022-04-19 12:00:00)         NaN         NaN   
[2022-04-19 12:00:00, 2022-04-19 13:00:00)         NaN         NaN   
[2022-04-19 13:00:00, 2022-04-19 14:00:00)         NaN         NaN   
[2022-04-19 14:00:00, 2022-04-19 15:00:00)  279.720001  282.420013   
[2022-04-19 15:00:00, 2022-04-19 16:00:00)  282.399994  284.750000   
[2022-04-19 16:00:00, 2022-04-19 17:00:00)  283.880005  284.820007   
[2022-04-19 17:00:00, 2022-04-19 18:00:00)  284.269989  284.269989   
[2022-04-19 18:00:00, 2022-04-19 19:00:00)  283.780304  284.059998   
[2022-04-19 19:00:00, 2022-04-19 20:00:00)  283.545013  284.359985   
[2022-04-19 20:00:00, 2022-04-19 21:00:00)  284.279999  286.170013   
[2022-04-20 08:00:00, 2022-04-20 09:00:00)         NaN         NaN   
[2022-04-20 09:00:00, 2022-04-20 10:00:00)         NaN         NaN   
[2022-04-20 10:00:00, 2022-04-20 11:00:00)         NaN         NaN   
[2022-04-20 11:00:00, 2022-04-20 12:00:00)         NaN         NaN   
[2022-04-20 12:00:00, 2022-04-20 13:00:00)         NaN         NaN   
[2022-04-20 13:00:00, 2022-04-20 14:00:00)         NaN         NaN   
[2022-04-20 14:00:00, 2022-04-20 15:00:00)  289.399994  289.700012   
[2022-04-20 15:00:00, 2022-04-20 16:00:00)  286.380005  288.890015   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-19 08:00:00, 2022-04-19 09:00:00)         NaN         NaN        NaN   
[2022-04-19 09:00:00, 2022-04-19 10:00:00)         NaN         NaN        NaN   
[2022-04-19 10:00:00, 2022-04-19 11:00:00)         NaN         NaN        NaN   
[2022-04-19 11:00:00, 2022-04-19 12:00:00)         NaN         NaN        NaN   
[2022-04-19 12:00:00, 2022-04-19 13:00:00)         NaN         NaN        NaN   
[2022-04-19 13:00:00, 2022-04-19 14:00:00)         NaN         NaN        NaN   
[2022-04-19 14:00:00, 2022-04-19 15:00:00)  278.410004  282.420013  3538466.0   
[2022-04-19 15:00:00, 2022-04-19 16:00:00)  282.059998  283.929993  3012724.0   
[2022-04-19 16:00:00, 2022-04-19 17:00:00)  283.600006  284.359985  2074591.0   
[2022-04-19 17:00:00, 2022-04-19 18:00:00)  283.290009  283.809998  1509603.0   
[2022-04-19 18:00:00, 2022-04-19 19:00:00)  282.980011  283.549988  1796292.0   
[2022-04-19 19:00:00, 2022-04-19 20:00:00)  282.910004  284.260010  1846124.0   
[2022-04-19 20:00:00, 2022-04-19 21:00:00)  284.049988  285.380005  4564482.0   
[2022-04-20 08:00:00, 2022-04-20 09:00:00)         NaN         NaN        NaN   
[2022-04-20 09:00:00, 2022-04-20 10:00:00)         NaN         NaN        NaN   
[2022-04-20 10:00:00, 2022-04-20 11:00:00)         NaN         NaN        NaN   
[2022-04-20 11:00:00, 2022-04-20 12:00:00)         NaN         NaN        NaN   
[2022-04-20 12:00:00, 2022-04-20 13:00:00)         NaN         NaN        NaN   
[2022-04-20 13:00:00, 2022-04-20 14:00:00)         NaN         NaN        NaN   
[2022-04-20 14:00:00, 2022-04-20 15:00:00)  285.980011  286.399994  3810859.0   
[2022-04-20 15:00:00, 2022-04-20 16:00:00)  285.799988  288.084991  3110928.0   

symbol                                             AZN.L               \
                                                    open         high   
[2022-04-19 08:00:00, 2022-04-19 09:00:00)  10452.000000  10550.00000   
[2022-04-19 09:00:00, 2022-04-19 10:00:00)  10490.000000  10519.78418   
[2022-04-19 10:00:00, 2022-04-19 11:00:00)  10458.000000  10490.00000   
[2022-04-19 11:00:00, 2022-04-19 12:00:00)  10472.000000  10530.00000   
[2022-04-19 12:00:00, 2022-04-19 13:00:00)  10500.000000  10530.00000   
[2022-04-19 13

As AZN.L is the `lead_symbol` prices are anchored on the 08:00 London open (NB the output has defaulted to local London time). The New York Stock Exchange usually opens at 14:30 London time (13:30 over the few weeks that DST observance is not synchronised). The NYSE opens fall midway through an indice (usually the 14:00 - 15:00 indices, 13:00 - 14:00 indices if DST is not synchronised).

If the same prices are requested with MSFT as the `lead_symbol` then each session's prices will instead be anchored on the New York opens.

In [62]:
df = prices_us_lon.get(
    "1H", start_session, end_session, lead_symbol="MSFT", tzout=xlon.tz
)
df

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 14:30:00, 2022-04-19 15:30:00)  279.720001  284.369385   
[2022-04-19 15:30:00, 2022-04-19 16:30:00)  283.359985  284.750000   
[2022-04-19 16:30:00, 2022-04-19 17:30:00)  284.410004  284.820007   
[2022-04-19 17:30:00, 2022-04-19 18:30:00)  283.600006  284.059998   
[2022-04-19 18:30:00, 2022-04-19 19:30:00)  283.179993  284.019989   
[2022-04-19 19:30:00, 2022-04-19 20:30:00)  283.519989  285.582092   
[2022-04-19 20:30:00, 2022-04-19 21:30:00)  285.279999  286.170013   
[2022-04-20 07:30:00, 2022-04-20 08:30:00)         NaN         NaN   
[2022-04-20 08:30:00, 2022-04-20 09:30:00)         NaN         NaN   
[2022-04-20 09:30:00, 2022-04-20 10:30:00)         NaN         NaN   
[2022-04-20 10:30:00, 2022-04-20 11:30:00)         NaN         NaN   
[2022-04-20 11:30:00, 2022-04-20 12:30:00)         NaN         NaN   
[2022-04-20 12:30:00, 2022-04-20 13:30:00)         NaN         NaN   
[2022-04-20 13:30:00, 2022-04-20 14:30:00)         NaN         NaN   
[2022-04-20 14:30:00, 2022-04-20 15:30:00)  289.399994  289.700012   
[2022-04-20 15:30:00, 2022-04-20 16:30:00)  288.149994  288.890015   
[2022-04-20 16:30:00, 2022-04-20 17:30:00)  286.519989  287.600006   
[2022-04-20 17:30:00, 2022-04-20 18:30:00)  285.740997  287.369995   
[2022-04-20 18:30:00, 2022-04-20 19:30:00)  286.880005  288.010010   
[2022-04-20 19:30:00, 2022-04-20 20:30:00)  286.959991  287.399994   
[2022-04-20 20:30:00, 2022-04-20 21:30:00)  287.109985  288.119904   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-19 14:30:00, 2022-04-19 15:30:00)  278.410004  283.329987  5277096.0   
[2022-04-19 15:30:00, 2022-04-19 16:30:00)  283.230011  284.420013  2510242.0   
[2022-04-19 16:30:00, 2022-04-19 17:30:00)  283.519989  283.587799  1537729.0   
[2022-04-19 17:30:00, 2022-04-19 18:30:00)  283.149994  283.200012  1598703.0   
[2022-04-19 18:30:00, 2022-04-19 19:30:00)  282.910004  283.519989  1975782.0   
[2022-04-19 19:30:00, 2022-04-19 20:30:00)  283.179993  285.250000  2159844.0   
[2022-04-19 20:30:00, 2022-04-19 21:30:00)  284.500000  285.380005  3282886.0   
[2022-04-20 07:30:00, 2022-04-20 08:30:00)         NaN         NaN        NaN   
[2022-04-20 08:30:00, 2022-04-20 09:30:00)         NaN         NaN        NaN   
[2022-04-20 09:30:00, 2022-04-20 10:30:00)         NaN         NaN        NaN   
[2022-04-20 10:30:00, 2022-04-20 11:30:00)         NaN         NaN        NaN   
[2022-04-20 11:30:00, 2022-04-20 12:30:00)         NaN         NaN        NaN   
[2022-04-20 12:30:00, 2022-04-20 13:30:00)         NaN         NaN        NaN   
[2022-04-20 13:30:00, 2022-04-20 14:30:00)         NaN         NaN        NaN   
[2022-04-20 14:30:00, 2022-04-20 15:30:00)  285.799988  288.100006  5723475.0   
[2022-04-20 15:30:00, 2022-04-20 16:30:00)  286.220001  286.510010  2621559.0   
[2022-04-20 16:30:00, 2022-04-20 17:30:00)  285.370209  285.750000  2143538.0   
[2022-04-20 17:30:00, 2022-04-20 18:30:00)  285.679993  286.929993  1644318.0   
[2022-04-20 18:30:00, 2022-04-20 19:30:00)  286.750000  286.950012  1613055.0   
[2022-04-20 19:30:00, 2022-04-20 20:30:00)  285.519989  287.029999  2230009.0   
[2022-04-20 20:30:00, 2022-04-20 21:30:00)  286.049988  286.309998  3366341.0   

symbol                                        AZN.L                    \
                                               open     high      low   
[2022-04-19 14:30:00, 2022-04-19 15:30:00)  10526.0  10580.0  10484.0   
[2022-04-19 15:30:00, 2022-04-19 16:30:00)  10582.0  10582.0  10502.0   
[2022-04-19 16:30:00, 2022-04-19 17:30:00)      NaN      NaN      NaN   
[2022-04-19 17:30:00, 2022-04-19 18:30:00)      NaN      NaN      NaN   
[2022-04-19 18:30:00, 2022-04-19 19:30:00)      NaN      NaN      NaN   
[2022-04-19 19

NB `tzout` was passed as the London timezone to provide for comparison with the earlier table. Also, as "MSFT" is now the lead symbol all indices prior to the New York open of the `start_session` are not included.

All indices now start at half-past an hour, aligned with the New York opens. Of note, the 08:00 London open of the second session now falls in the 07:30 - 08:30 indice, i.e. to maintain the interval based on the NYSE open, it's necessary to introduce a non-trading period, from 07:30 - 08:00, prior to the London open.

In [63]:
df.pt.indices_partial_trading_info(xlon)

{Interval('2022-04-20 07:30:00', '2022-04-20 08:30:00', closed='left'): IntervalIndex([[2022-04-20 07:30:00, 2022-04-20 08:00:00)], dtype='interval[datetime64[ns, Europe/London], left]')}

`force` can be passed to force the left side of such indices to the first time that any symbol traded, in this case to the London 08:00 open.

In [64]:
prices_us_lon.get(
    "1H", start_session, end_session, lead_symbol="MSFT", tzout=xlon.tz, force=True,
).iloc[6:9]  # showing only relevant rows...

symbol                                            MSFT                     \
                                                  open        high    low   
[2022-04-19 20:30:00, 2022-04-19 21:00:00)  285.279999  286.170013  284.5   
[2022-04-20 08:00:00, 2022-04-20 08:30:00)         NaN         NaN    NaN   
[2022-04-20 08:30:00, 2022-04-20 09:30:00)         NaN         NaN    NaN   

symbol                                                               AZN.L  \
                                                 close     volume     open   
[2022-04-19 20:30:00, 2022-04-19 21:00:00)  285.380005  3282886.0      NaN   
[2022-04-20 08:00:00, 2022-04-20 08:30:00)         NaN        NaN  10394.0   
[2022-04-20 08:30:00, 2022-04-20 09:30:00)         NaN        NaN  10416.0   

symbol                                                                 \
                                               high      low    close   
[2022-04-19 20:30:00, 2022-04-19 21:00:00)      NaN      NaN      NaN   
[2022-04-20 08:00:00, 2022-04-20 08:30:00)  10482.0  10392.0  10418.0   
[2022-04-20 08:30:00, 2022-04-20 09:30:00)  10480.0  10410.0  10474.0   

symbol                                                
                                              volume  
[2022-04-19 20:30:00, 2022-04-19 21:00:00)       NaN  
[2022-04-20 08:00:00, 2022-04-20 08:30:00)   61124.0  
[2022-04-20 08:30:00, 2022-04-20 09:30:00)  104366.0

#### `.pt` accessor methods

This subsection looks at how `.pt` accessor methods can be used to interogate indices of prices tables for symbols that are not all associated with the same calendar.

Some `.pt` accessor methods take an argument to define the calendar against which to evaluate the return. This is demonstrated below using the 'unforced' example that has MSFT as the lead symbol.

In [65]:
df.pt.indices_partial_trading(xlon)

IntervalIndex([[2022-04-20 07:30:00, 2022-04-20 08:30:00)], dtype='interval[datetime64[ns, Europe/London], left]')

Passing `.pt.indices_partial_trading` the London calendar (xlon) associated with AstraZeneca returns the single indice that covers the 08:00 xlon open of the second session. Of all the table's indices this is the only one that, when evaluated against the London calendar, partly covers a non-trading period and partly covers a trading period (all other indices cover periods during all of which the London exchange was either open or closed).

In contrast, if the method is passed the New York calendar associated with Microsoft (xnys)...

In [66]:
df.pt.indices_partial_trading(xnys)

IntervalIndex([[2022-04-19 20:30:00, 2022-04-19 21:30:00), [2022-04-20 20:30:00, 2022-04-20 21:30:00)], dtype='interval[datetime64[ns, Europe/London], left]')

There are two partial indices. These represent the unaligned New York closes. The actual close times for these sessions are (in London time)...

In [67]:
(
    xnys.session_close(start_session).astimezone(xlon.tz),
    xnys.session_close(end_session).astimezone(xlon.tz),
)

(Timestamp('2022-04-19 21:00:00+0100', tz='Europe/London'),
 Timestamp('2022-04-20 21:00:00+0100', tz='Europe/London'))

All other indices cover periods during all of which New York was either open or closed.

`.pt.indices_non_trading` can be used to query those indices covering periods during all of which the New York exchange was closed.

In [68]:
df.pt.indices_non_trading(xnys)

IntervalIndex([[2022-04-20 07:30:00, 2022-04-20 08:30:00), [2022-04-20 08:30:00, 2022-04-20 09:30:00), [2022-04-20 09:30:00, 2022-04-20 10:30:00), [2022-04-20 10:30:00, 2022-04-20 11:30:00), [2022-04-20 11:30:00, 2022-04-20 12:30:00), [2022-04-20 12:30:00, 2022-04-20 13:30:00), [2022-04-20 13:30:00, 2022-04-20 14:30:00)], dtype='interval[datetime64[ns, Europe/London], left]')

Similarly, to get those indices covering periods during which London was closed...

In [69]:
df.pt.indices_non_trading(xlon)

IntervalIndex([[2022-04-19 16:30:00, 2022-04-19 17:30:00), [2022-04-19 17:30:00, 2022-04-19 18:30:00), [2022-04-19 18:30:00, 2022-04-19 19:30:00), [2022-04-19 19:30:00, 2022-04-19 20:30:00), [2022-04-19 20:30:00, 2022-04-19 21:30:00), [2022-04-20 16:30:00, 2022-04-20 17:30:00), [2022-04-20 17:30:00, 2022-04-20 18:30:00), [2022-04-20 18:30:00, 2022-04-20 19:30:00), [2022-04-20 19:30:00, 2022-04-20 20:30:00), [2022-04-20 20:30:00, 2022-04-20 21:30:00)], dtype='interval[datetime64[ns, Europe/London], left]')

`.pt.indices_trading` will query those indices that cover periods during all of which the passed calendar is open.

The following returns those indices during all of which both xlon and xnys were open.

In [70]:
df.pt.indices_trading(xnys).intersection(df.pt.indices_trading(xlon))

IntervalIndex([[2022-04-19 14:30:00, 2022-04-19 15:30:00), [2022-04-19 15:30:00, 2022-04-19 16:30:00), [2022-04-20 14:30:00, 2022-04-20 15:30:00), [2022-04-20 15:30:00, 2022-04-20 16:30:00)], dtype='interval[datetime64[ns, Europe/London], left]')

During all other periods either one of the calendars was closed either all or some of the time.

`.pt.indices_trading_status` can be used to get a snapshot of the trading status of each indice. This was shown in earlier examples for a single symbol. The method's particularly useful for multiple symbols (associated with different calendars) as False indicates those indices during which the passed calendar was always closed.

In [71]:
df.pt.indices_trading_status(xnys)

[2022-04-19 14:30:00, 2022-04-19 15:30:00)     True
[2022-04-19 15:30:00, 2022-04-19 16:30:00)     True
[2022-04-19 16:30:00, 2022-04-19 17:30:00)     True
[2022-04-19 17:30:00, 2022-04-19 18:30:00)     True
[2022-04-19 18:30:00, 2022-04-19 19:30:00)     True
[2022-04-19 19:30:00, 2022-04-19 20:30:00)     True
[2022-04-19 20:30:00, 2022-04-19 21:30:00)      NaN
[2022-04-20 07:30:00, 2022-04-20 08:30:00)    False
[2022-04-20 08:30:00, 2022-04-20 09:30:00)    False
[2022-04-20 09:30:00, 2022-04-20 10:30:00)    False
[2022-04-20 10:30:00, 2022-04-20 11:30:00)    False
[2022-04-20 11:30:00, 2022-04-20 12:30:00)    False
[2022-04-20 12:30:00, 2022-04-20 13:30:00)    False
[2022-04-20 13:30:00, 2022-04-20 14:30:00)    False
[2022-04-20 14:30:00, 2022-04-20 15:30:00)     True
[2022-04-20 15:30:00, 2022-04-20 16:30:00)     True
[2022-04-20 16:30:00, 2022-04-20 17:30:00)     True
[2022-04-20 17:30:00, 2022-04-20 18:30:00)     True
[2022-04-20 18:30:00, 2022-04-20 19:30:00)     True
[2022-04-20 

Recall that True indicates those indices that cover periods during all of which the passed calendar was open and a missing value (NaN) represents partial trading indices that cover periods during which the calendar was both open and closed. In the above example the False indices are therefore those that cover periods during which only the London market was open.

`.pt.indices_trading_minutes` will show the number of trading minutes that each indices represents, with those trading minutes evaluated against the passed calendar.

In [72]:
df.pt.indices_trading_minutes(xnys)

[2022-04-19 14:30:00, 2022-04-19 15:30:00)    60
[2022-04-19 15:30:00, 2022-04-19 16:30:00)    60
[2022-04-19 16:30:00, 2022-04-19 17:30:00)    60
[2022-04-19 17:30:00, 2022-04-19 18:30:00)    60
[2022-04-19 18:30:00, 2022-04-19 19:30:00)    60
[2022-04-19 19:30:00, 2022-04-19 20:30:00)    60
[2022-04-19 20:30:00, 2022-04-19 21:30:00)    30
[2022-04-20 07:30:00, 2022-04-20 08:30:00)     0
[2022-04-20 08:30:00, 2022-04-20 09:30:00)     0
[2022-04-20 09:30:00, 2022-04-20 10:30:00)     0
[2022-04-20 10:30:00, 2022-04-20 11:30:00)     0
[2022-04-20 11:30:00, 2022-04-20 12:30:00)     0
[2022-04-20 12:30:00, 2022-04-20 13:30:00)     0
[2022-04-20 13:30:00, 2022-04-20 14:30:00)     0
[2022-04-20 14:30:00, 2022-04-20 15:30:00)    60
[2022-04-20 15:30:00, 2022-04-20 16:30:00)    60
[2022-04-20 16:30:00, 2022-04-20 17:30:00)    60
[2022-04-20 17:30:00, 2022-04-20 18:30:00)    60
[2022-04-20 18:30:00, 2022-04-20 19:30:00)    60
[2022-04-20 19:30:00, 2022-04-20 20:30:00)    60
[2022-04-20 20:30:00

In [73]:
df.pt.indices_trading_minutes(xlon)

[2022-04-19 14:30:00, 2022-04-19 15:30:00)    60
[2022-04-19 15:30:00, 2022-04-19 16:30:00)    60
[2022-04-19 16:30:00, 2022-04-19 17:30:00)     0
[2022-04-19 17:30:00, 2022-04-19 18:30:00)     0
[2022-04-19 18:30:00, 2022-04-19 19:30:00)     0
[2022-04-19 19:30:00, 2022-04-19 20:30:00)     0
[2022-04-19 20:30:00, 2022-04-19 21:30:00)     0
[2022-04-20 07:30:00, 2022-04-20 08:30:00)    30
[2022-04-20 08:30:00, 2022-04-20 09:30:00)    60
[2022-04-20 09:30:00, 2022-04-20 10:30:00)    60
[2022-04-20 10:30:00, 2022-04-20 11:30:00)    60
[2022-04-20 11:30:00, 2022-04-20 12:30:00)    60
[2022-04-20 12:30:00, 2022-04-20 13:30:00)    60
[2022-04-20 13:30:00, 2022-04-20 14:30:00)    60
[2022-04-20 14:30:00, 2022-04-20 15:30:00)    60
[2022-04-20 15:30:00, 2022-04-20 16:30:00)    60
[2022-04-20 16:30:00, 2022-04-20 17:30:00)     0
[2022-04-20 17:30:00, 2022-04-20 18:30:00)     0
[2022-04-20 18:30:00, 2022-04-20 19:30:00)     0
[2022-04-20 19:30:00, 2022-04-20 20:30:00)     0
[2022-04-20 20:30:00

#### Calendars with breaks

Towards the start of this tutorial it was stated that where an exchange has a break the indices for the morning session are anchored on the morning subsession open and indices for the afternoon session are anchored on the afternoon subsession open. This was shown with the example of Alibaba's Hong Kong listing, where the market observes a lunch break from 12:00 - 13:00.

In [74]:
df = prices_hk.get("1H", session, session, anchor="open")
df

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-20 09:30:00, 2022-04-20 10:30:00)  91.199997  91.900002  90.599998   
[2022-04-20 10:30:00, 2022-04-20 11:30:00)  90.800003  92.199997  90.500000   
[2022-04-20 11:30:00, 2022-04-20 12:30:00)  92.000000  92.949997  91.949997   
[2022-04-20 13:00:00, 2022-04-20 14:00:00)  92.650002  92.750000  91.050003   
[2022-04-20 14:00:00, 2022-04-20 15:00:00)  91.150002  91.349998  90.300003   
[2022-04-20 15:00:00, 2022-04-20 16:00:00)  90.650002  91.300003  90.550003   

symbol                                                            
                                                close     volume  
[2022-04-20 09:30:00, 2022-04-20 10:30:00)  90.800003  3261815.0  
[2022-04-20 10:30:00, 2022-04-20 11:30:00)  92.000000  3225833.0  
[2022-04-20 11:30:00, 2022-04-20 12:30:00)  92.599998  1359203.0  
[2022-04-20 13:00:00, 2022-04-20 14:00:00)  91.150002  2424843.0  
[2022-04-20 14:00:00, 2022-04-20 15:00:00)  90.650002  2740490.0  
[2022-04-20 15:00:00, 2022-04-20 16:00:00)  90.650002  4127142.0

Notice that at this one hour interval the last indice of the am subsession is unaligned with the morning session's 12:00 close.

In [75]:
df.pt.indices_partial_trading_info(xhkg)

{Interval('2022-04-20 11:30:00', '2022-04-20 12:30:00', closed='left'): IntervalIndex([[2022-04-20 12:00:00, 2022-04-20 12:30:00)], dtype='interval[datetime64[ns, Asia/Hong_Kong], left]')}

The afternoon session then opens at 13:00.

It's not always the case that indices of an afternoon subsession will be anchored on the afternoon open. In the following cases **all** indices may be anchored on the morning subsession open.

* Prices from the data source do not recognise the break (rather, all prices for a session are anchored on the morning open).

* An indice of a trading index evaluated against the calendar that observes the break *partially* overlaps an indice of a trading index evaluated against another calendar.

##### **Data source does not observe the break**

Yahoo, the default data provider, does not recognise the Hong Kong break. Rather data at one hour looks like this...

In [76]:
print(session_h1)
prices_hk.get("1H", session_h1, session_h1, anchor="open")

2020-05-15 00:00:00


symbol                                         9988.HK              \
                                                  open        high   
[2020-05-15 09:30:00, 2020-05-15 10:30:00)  195.399994  196.100006   
[2020-05-15 10:30:00, 2020-05-15 11:30:00)  194.500000  195.800003   
[2020-05-15 11:30:00, 2020-05-15 12:30:00)  195.800003  196.000000   
[2020-05-15 12:30:00, 2020-05-15 13:30:00)  196.000000  196.699997   
[2020-05-15 13:30:00, 2020-05-15 14:30:00)  196.500000  196.699997   
[2020-05-15 14:30:00, 2020-05-15 15:30:00)  196.500000  197.100006   
[2020-05-15 15:30:00, 2020-05-15 16:30:00)  196.800003  197.300003   

symbol                                                                       
                                                   low       close   volume  
[2020-05-15 09:30:00, 2020-05-15 10:30:00)  194.100006  194.500000        0  
[2020-05-15 10:30:00, 2020-05-15 11:30:00)  194.300003  195.800003  1430128  
[2020-05-15 11:30:00, 2020-05-15 12:30:00)  195.399994  196.000000   663560  
[2020-05-15 12:30:00, 2020-05-15 13:30:00)  195.899994  196.500000  3349692  
[2020-05-15 13:30:00, 2020-05-15 14:30:00)  196.000000  196.500000  4604192  
[2020-05-15 14:30:00, 2020-05-15 15:30:00)  196.199997  196.800003  5395882  
[2020-05-15 15:30:00, 2020-05-15 16:30:00)  196.600006  196.899994  6117562

Notice how the prices just plough through the 12:00 - 13:00 break as if it weren't there. But hold on, what's different between this example and the one further above that recognised the break with the same 1H interval! The difference is that the earlier example is for a session for which price data is available at 5 minute intervals. If the available data allows, `market_prices` will downsample data at a lower interval in order to maintain the break. The example here however is requesting data for `session_h1` which is a session for which intraday data is only available at hourly intervals.

Although the indices aren't anchored on the afternoon open, forcing will still curtail the indices such that they only cover trading periods.

In [77]:
prices_hk.get("1H", session_h1, session_h1, anchor="open", force=True)

symbol                                         9988.HK              \
                                                  open        high   
[2020-05-15 09:30:00, 2020-05-15 10:30:00)  195.399994  196.100006   
[2020-05-15 10:30:00, 2020-05-15 11:30:00)  194.500000  195.800003   
[2020-05-15 11:30:00, 2020-05-15 12:00:00)  195.800003  196.000000   
[2020-05-15 13:00:00, 2020-05-15 13:30:00)  196.000000  196.699997   
[2020-05-15 13:30:00, 2020-05-15 14:30:00)  196.500000  196.699997   
[2020-05-15 14:30:00, 2020-05-15 15:30:00)  196.500000  197.100006   
[2020-05-15 15:30:00, 2020-05-15 16:00:00)  196.800003  197.300003   

symbol                                                                       
                                                   low       close   volume  
[2020-05-15 09:30:00, 2020-05-15 10:30:00)  194.100006  194.500000        0  
[2020-05-15 10:30:00, 2020-05-15 11:30:00)  194.300003  195.800003  1430128  
[2020-05-15 11:30:00, 2020-05-15 12:00:00)  195.399994  196.000000   663560  
[2020-05-15 13:00:00, 2020-05-15 13:30:00)  195.899994  196.500000  3349692  
[2020-05-15 13:30:00, 2020-05-15 14:30:00)  196.000000  196.500000  4604192  
[2020-05-15 14:30:00, 2020-05-15 15:30:00)  196.199997  196.800003  5395882  
[2020-05-15 15:30:00, 2020-05-15 16:00:00)  196.600006  196.899994  6117562

Notice that the first indice of the afternoon session now only covers half an hour rather than the hour interval requested, i.e. forcing simply alters the indices that are there. The afternoon indices are still anchored on the morning open, not the afternoon open.

##### **Indices of trading indexes of different calendars overlap**

The short version is that if the union of trading indexes of each associated calendar has at least one indice that partially overlaps another then any breaks will be ignored and all indices will all be anchored on the sessions' opens.

The longer version follows (the rest of this cell can be safely ignored if you're not interested in what's going on under-the-bonnet)...

`market_prices` gets prices from the data provider at intervals defined as `base intervals`. Collectively these base intervals provide for requesting all data that the provider makes available. Usually requests for prices at a base interval will be met by simply serving the data as provided by the source. For all other intervals, price data at a suitable base interval is downsampled to the requested interval. There are two ways that `market_prices` downsamples intraday data. The preference is to do this:-
* For each calendar associated with the symbols a trading index is created at the request interval. A union of these indexes is then evaluated and its indices treated as 'bins'. Data from the base interval is then binned to the corresponding downsampled indice. It's quick and accurate, but only works if the union of the indexes contains no indice that partially overlaps another. Accordingly, it will always work when the trading indexes do not overlap (such as Hong Kong stocks and US stocks), and when the indexes do overlap although the indices fully align. Indices usually fully align for low regular intervals like 10T, 15T, 20T, 30T. Less regular intervals such as 7T, 25T, 40T are more likely to result in the indices of different indexes partially overlapping, as are high regular intervals such as 1H and higher.

If the above approach is not possible (because indices of the unified index partially overlap) then `market_prices` 'simply' downsamples the base data and for each assumed session evaluates indices that are anchored to the open of the lead calendar (this is implemented by calling `.pt.downsample` on a subset of the base data). The downsampled data has no awareness of any breaks.

The following offers an example using prices that include stocks trading in London and Hong Kong. Unifiying the trading indexes for the xlon and xhkg calendars results in partial overlaps at certain intervals.

During the summer, whilst DST is observed in London, London opens one hour prior to the Hong Kong close. During the winter London opens at the same time that Hong Kong closes. However, even when the Hong Kong close and London open align the trading indexes for the two exchanges can conflict - whenever the interval is such that the last Hong Kong indice of a session does not align with the Hong Kong close it will overlap with the first indice of the London session.

Indices of the trading indexes for xlon and xhkg overlap at a 40 minute interval...

In [78]:
prices_hk_lon = PricesYahoo("9988.HK, AZN.L")
df = prices_hk_lon.get("50T", start_session, end_session, lead_symbol="9988.HK", anchor="open")
df.iloc[:11]  # only show up to the most relevant part

symbol                                        9988.HK                        \
                                                 open       high        low   
[2022-04-19 09:30:00, 2022-04-19 10:20:00)  92.650002  93.449997  91.349998   
[2022-04-19 10:20:00, 2022-04-19 11:10:00)  91.800003  91.949997  91.199997   
[2022-04-19 11:10:00, 2022-04-19 12:00:00)  91.650002  92.699997  91.599998   
[2022-04-19 12:50:00, 2022-04-19 13:40:00)  92.150002  92.750000  91.949997   
[2022-04-19 13:40:00, 2022-04-19 14:30:00)  92.150002  92.150002  91.250000   
[2022-04-19 14:30:00, 2022-04-19 15:20:00)  91.550003  91.900002  91.099998   
[2022-04-19 15:20:00, 2022-04-19 16:10:00)  91.300003  91.849998  90.900002   
[2022-04-19 16:10:00, 2022-04-19 17:00:00)        NaN        NaN        NaN   
[2022-04-19 17:00:00, 2022-04-19 17:50:00)        NaN        NaN        NaN   
[2022-04-19 17:50:00, 2022-04-19 18:40:00)        NaN        NaN        NaN   
[2022-04-19 18:40:00, 2022-04-19 19:30:00)        NaN        NaN        NaN   

symbol                                                            \
                                                close     volume   
[2022-04-19 09:30:00, 2022-04-19 10:20:00)  91.800003  6318190.0   
[2022-04-19 10:20:00, 2022-04-19 11:10:00)  91.599998  3464268.0   
[2022-04-19 11:10:00, 2022-04-19 12:00:00)  92.150002  3243483.0   
[2022-04-19 12:50:00, 2022-04-19 13:40:00)  92.199997  2180445.0   
[2022-04-19 13:40:00, 2022-04-19 14:30:00)  91.550003  3792521.0   
[2022-04-19 14:30:00, 2022-04-19 15:20:00)  91.300003  3923523.0   
[2022-04-19 15:20:00, 2022-04-19 16:10:00)  91.400002  5742434.0   
[2022-04-19 16:10:00, 2022-04-19 17:00:00)        NaN        NaN   
[2022-04-19 17:00:00, 2022-04-19 17:50:00)        NaN        NaN   
[2022-04-19 17:50:00, 2022-04-19 18:40:00)        NaN        NaN   
[2022-04-19 18:40:00, 2022-04-19 19:30:00)        NaN        NaN   

symbol                                             AZN.L               \
                                                    open         high   
[2022-04-19 09:30:00, 2022-04-19 10:20:00)           NaN          NaN   
[2022-04-19 10:20:00, 2022-04-19 11:10:00)           NaN          NaN   
[2022-04-19 11:10:00, 2022-04-19 12:00:00)           NaN          NaN   
[2022-04-19 12:50:00, 2022-04-19 13:40:00)           NaN          NaN   
[2022-04-19 13:40:00, 2022-04-19 14:30:00)           NaN          NaN   
[2022-04-19 14:30:00, 2022-04-19 15:20:00)  10452.000000  10550.00000   
[2022-04-19 15:20:00, 2022-04-19 16:10:00)  10502.000000  10534.00000   
[2022-04-19 16:10:00, 2022-04-19 17:00:00)  10503.099609  10519.78418   
[2022-04-19 17:00:00, 2022-04-19 17:50:00)  10458.000000  10490.00000   
[2022-04-19 17:50:00, 2022-04-19 18:40:00)  10456.000000  10530.00000   
[2022-04-19 18:40:00, 2022-04-19 19:30:00)  10522.000000  10526.00000   

symbol                                                                       
                                                low         close    volume  
[2022-04-19 09:30:00, 2022-04-19 10:20:00)      NaN           NaN       NaN  
[2022-04-19 10:20:00, 2022-04-19 11:10:00)      NaN           NaN       NaN  
[2022-04-19 11:10:00, 2022-04-19 12:00:00)      NaN           NaN       NaN  
[2022-04-19 12:50:00, 2022-04-19 13:40:00)      NaN           NaN       NaN  
[2022-04-19 13:40:00, 2022-04-19 14:30:00)      NaN           NaN       NaN  
[2022-04-19 14:30:00, 2022-04-19 15:20:00)  10430.0  10502.780273   50548.0  
[2022-04-19 15:20:00, 2022-04-19 16:10:00)  10478.0  10512.000000   93459.0  
[2022-04-19 16:10:00, 2022-04-19 17:00:00)  10454.0  10458.839844   51744.0  
[2022-04-19 17:00:00, 2022-04-19 17:50:00)  10428.0  10454.000000   77563.0  
[2022-04-19 17:50:00, 2022-04-19 18:40:00)  10450.0  10522.200195   71477.0  
[2022-04-19 18:40:00, 2022-04-19 19:30:00)  10478.0  10512.000000  538050.0

The first indice of the afternoon session starts at 12:50, i.e. 10 minutes before the afternoon open. This is because the indice is anchored on the morning open. The indice from 12:00 - 12:50 is excluded simply because there would be no price data for this indice (neither exchange is open during this period).

As with the example for the data source not observing the break, `force` could be passed so that the first indice of the afternoon session starts at 13:00, although this indice would only cover 40 minutes (albeit all of which would be trading minutes).

### `anchor` "workback" with multiple calendars

Anchoring as "workback" is pretty much the same for symbols associated with different calendars as it is for [symbols associated with a single calendars](#anchor-"workback"). The indices are still evaluated based on a regular number of trading minutes, the only difference being that the 'trading minutes' are evaluted as trading minutes of **any** of the associated calendars.

In [79]:
df = prices_us_lon.get("2H", start_session, end_session, lead_symbol="MSFT", anchor="workback")
df

symbol                                            MSFT              \
                                                  open        high   
[2022-04-19 11:00:00, 2022-04-19 13:00:00)  283.880005  284.820007   
[2022-04-19 13:00:00, 2022-04-19 15:00:00)  283.780304  284.359985   
[2022-04-19 15:00:00, 2022-04-20 04:00:00)  284.279999  286.170013   
[2022-04-20 04:00:00, 2022-04-20 06:00:00)         NaN         NaN   
[2022-04-20 06:00:00, 2022-04-20 08:00:00)         NaN         NaN   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  289.399994  289.700012   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  286.380005  288.890015   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  287.209991  288.010010   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  287.549988  288.119904   

symbol                                                                         \
                                                   low       close     volume   
[2022-04-19 11:00:00, 2022-04-19 13:00:00)  283.290009  283.809998  3584194.0   
[2022-04-19 13:00:00, 2022-04-19 15:00:00)  282.910004  284.260010  3642416.0   
[2022-04-19 15:00:00, 2022-04-20 04:00:00)  284.049988  285.380005  4564482.0   
[2022-04-20 04:00:00, 2022-04-20 06:00:00)         NaN         NaN        NaN   
[2022-04-20 06:00:00, 2022-04-20 08:00:00)         NaN         NaN        NaN   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  285.980011  286.399994  3810859.0   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  285.799988  287.170013  5632208.0   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  285.370209  287.570007  3479485.0   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  285.519989  286.309998  6419743.0   

symbol                                        AZN.L                    \
                                               open     high      low   
[2022-04-19 11:00:00, 2022-04-19 13:00:00)  10520.0  10530.0  10502.0   
[2022-04-19 13:00:00, 2022-04-19 15:00:00)      NaN      NaN      NaN   
[2022-04-19 15:00:00, 2022-04-20 04:00:00)  10394.0  10482.0  10392.0   
[2022-04-20 04:00:00, 2022-04-20 06:00:00)  10448.0  10496.0  10394.0   
[2022-04-20 06:00:00, 2022-04-20 08:00:00)  10488.0  10494.0  10394.0   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  10436.0  10450.0  10392.0   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  10416.0  10490.0  10394.0   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)      NaN      NaN      NaN   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)      NaN      NaN      NaN   

symbol                                                               
                                                   close     volume  
[2022-04-19 11:00:00, 2022-04-19 13:00:00)  10506.000000   100242.0  
[2022-04-19 13:00:00, 2022-04-19 15:00:00)           NaN        NaN  
[2022-04-19 15:00:00, 2022-04-20 04:00:00)  10444.000000   119879.0  
[2022-04-20 04:00:00, 2022-04-20 06:00:00)  10488.000000  1177311.0  
[2022-04-20 06:00:00, 2022-04-20 08:00:00)  10436.900391   152432.0  
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  10418.000000   650291.0  
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  10468.000000   723708.0  
[2022-04-20 12:00:00, 2022-04-20 14:00:00)           NaN        NaN  
[2022-04-20 14:00:00, 2022-04-20 16:00:00)           NaN        NaN

The right of the last indice is anchored to the evaluated period end (end of the second MSFT session) from where indices are evaluated backwards such that they each comprise 2 hours of trading minutes, regardless of whether those minutes relate to a period when the London market was open, when the New York market was open, or when both markets were open.

In [80]:
df.pt.indices_trading_minutes(xlon)

[2022-04-19 11:00:00, 2022-04-19 13:00:00)     30
[2022-04-19 13:00:00, 2022-04-19 15:00:00)      0
[2022-04-19 15:00:00, 2022-04-20 04:00:00)     60
[2022-04-20 04:00:00, 2022-04-20 06:00:00)    120
[2022-04-20 06:00:00, 2022-04-20 08:00:00)    120
[2022-04-20 08:00:00, 2022-04-20 10:00:00)    120
[2022-04-20 10:00:00, 2022-04-20 12:00:00)     90
[2022-04-20 12:00:00, 2022-04-20 14:00:00)      0
[2022-04-20 14:00:00, 2022-04-20 16:00:00)      0
Name: trading_mins, dtype: int64

In [81]:
df.pt.indices_trading_minutes(xnys)

[2022-04-19 11:00:00, 2022-04-19 13:00:00)    120
[2022-04-19 13:00:00, 2022-04-19 15:00:00)    120
[2022-04-19 15:00:00, 2022-04-20 04:00:00)     60
[2022-04-20 04:00:00, 2022-04-20 06:00:00)      0
[2022-04-20 06:00:00, 2022-04-20 08:00:00)      0
[2022-04-20 08:00:00, 2022-04-20 10:00:00)     30
[2022-04-20 10:00:00, 2022-04-20 12:00:00)    120
[2022-04-20 12:00:00, 2022-04-20 14:00:00)    120
[2022-04-20 14:00:00, 2022-04-20 16:00:00)    120
Name: trading_mins, dtype: int64

### `openend`

The `opened` parameter was introduced [earlier in this tutorial](#Force-to-period-end-with-openend). The option is further explored here in the context of symbols associated with multiple calendars, specifically Bitcoin and the UK listing of the equity AstraZeneca.

In [82]:
prices_uk_247 = PricesYahoo(["AZN.L", "BTC-USD"], lead_symbol="AZN.L")

Bitcoin trades continuously 24/7 whilst AstraZeneca trades on the London Stock Exchange. The following cell shows the local open and close for a specific session of the LSE.

In [83]:
start, end = xlon.session_open_close(session)
# put in terms of local tz for ease of reference
start = start.astimezone(prices_uk_247.tz_default)
end = end.astimezone(prices_uk_247.tz_default)
# for reference
start, end

(Timestamp('2022-04-20 08:00:00+0100', tz='Europe/London'),
 Timestamp('2022-04-20 16:30:00+0100', tz='Europe/London'))

Let's get prices at a 2 hour interval between these session bounds...

In [84]:
prices_uk_247.get("2H", start, end)

symbol                                             AZN.L                    \
                                                    open     high      low   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  10394.000000  10482.0  10392.0   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  10450.000000  10496.0  10394.0   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  10456.908203  10458.0  10412.0   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  10444.000000  10490.0  10392.0   

symbol                                                               BTC-USD  \
                                              close     volume          open   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  10448.0    97362.0  41444.296875   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  10456.0  1171999.0  41489.980469   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  10442.0   112627.0  41801.343750   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  10458.0  1240866.0  42117.835938   

symbol                                                                  \
                                                    high           low   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  41526.152344  41332.937500   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  41822.742188  41473.695312   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  42126.300781  41745.746094   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  42118.371094  41571.230469   

symbol                                                                  
                                                   close        volume  
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  41476.156250  4.894884e+08  
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  41795.542969  1.266676e+09  
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  42119.339844  1.451430e+09  
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  41581.796875  0.000000e+00

Although the session closes at 16.30, the last indice only offers prices through to 16:00, i.e. prices for the last half hour of the trading day are not included.

As in the [earlier example]((#Force-to-period-end-with-openend), the period end is an unaligned session close. Although unlike the earlier example, where the right side of the last indice fell after the session close, here the right side of the last indice falls before the session close. The reason for the difference is that including a further indice here, from 16:00 through 18:00, would result in the inclusion of prices for Bitcoin from 16.30 through 18.00, and in accordance with the Golden Rule, no prices are ever included before the period start or after the period end.

Both examples show how the default `openend` option, **"maintain"**, ensures **the interval is maintained when the period end is an unaligned session close**. The difference in the two examples is whether the right side of the final indice fell after or before the session close.
* If no symbol trades in the period betweeen the unaligned session close and the right of the indice that contains the session close then the right side of the final indice will fall after the session close (as in the earlier example for the single symbol "MSFT").
* If any symbol trades in the period betweeen the unaligned session close and the right of the indice that contains the session close then the final indice will be the latest indice with a right side that preceeds the session close, i.e. the right of the final indice will fall to the left of the session close, as in the example above.

As in the earlier example, if reflecting the period end is more important than maintaining the interval then `openend` can be passed as **"shorten"** to shorten the final indice so that it runs to the period end.

In [85]:
df = prices_uk_247.get("2H", start, end, openend="shorten")
df

symbol                                             AZN.L                    \
                                                    open     high      low   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  10394.000000  10482.0  10392.0   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  10450.000000  10496.0  10394.0   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  10456.908203  10458.0  10412.0   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  10444.000000  10490.0  10392.0   
[2022-04-20 16:00:00, 2022-04-20 16:30:00)  10458.242188  10470.0  10436.0   

symbol                                                               BTC-USD  \
                                              close     volume          open   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  10448.0   217241.0  41444.296875   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  10456.0  1171999.0  41489.980469   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  10442.0   112627.0  41801.343750   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  10458.0  1240866.0  42117.835938   
[2022-04-20 16:00:00, 2022-04-20 16:30:00)  10468.0    80888.0  41577.878906   

symbol                                                                  \
                                                    high           low   
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  41526.152344  41332.937500   
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  41822.742188  41473.695312   
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  42126.300781  41745.746094   
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  42118.371094  41571.230469   
[2022-04-20 16:00:00, 2022-04-20 16:30:00)  41577.878906  41141.402344   

symbol                                                                  
                                                   close        volume  
[2022-04-20 08:00:00, 2022-04-20 10:00:00)  41476.156250  5.860311e+08  
[2022-04-20 10:00:00, 2022-04-20 12:00:00)  41795.542969  1.366741e+09  
[2022-04-20 12:00:00, 2022-04-20 14:00:00)  42119.339844  1.510134e+09  
[2022-04-20 14:00:00, 2022-04-20 16:00:00)  41581.796875  3.350876e+08  
[2022-04-20 16:00:00, 2022-04-20 16:30:00)  41158.550781  3.353395e+08

In [86]:
df.pt.indices_length

0 days 02:00:00    4
0 days 00:30:00    1
dtype: int64

However, defining a shorter final interval is only possible if underlying base data is available at an interval that's sufficiently granular to evalaute that shorter indice. In the above example the underlying data, from which the table is evaluated, has a 5 minute interval. Look at what happens if the example is repeated for a session that's far enough back that the only base data available has a 1 hour interval.

In [87]:
start, end = xlon.session_open_close(session_h1)
# put in terms of local tz for ease of reference
start = start.astimezone(prices_uk_247.tz_default)
end = end.astimezone(prices_uk_247.tz_default)
# for reference
start, end

(Timestamp('2020-05-15 08:00:00+0100', tz='Europe/London'),
 Timestamp('2020-05-15 16:30:00+0100', tz='Europe/London'))

In [88]:
prices_uk_247.get("2H", start, end, openend="shorten")

symbol                                       AZN.L                          \
                                              open    high     low   close   
[2020-05-15 08:00:00, 2020-05-15 10:00:00)  8740.0  8796.0  8625.0  8641.0   
[2020-05-15 10:00:00, 2020-05-15 12:00:00)  8638.0  8659.0  8603.0  8637.0   
[2020-05-15 12:00:00, 2020-05-15 14:00:00)  8612.0  8676.0  8536.0  8601.0   
[2020-05-15 14:00:00, 2020-05-15 16:00:00)  8665.0  8732.0  8536.0  8660.0   

symbol                                                    BTC-USD  \
                                              volume         open   
[2020-05-15 08:00:00, 2020-05-15 10:00:00)  489577.0  9611.995117   
[2020-05-15 10:00:00, 2020-05-15 12:00:00)  711982.0  9607.218750   
[2020-05-15 12:00:00, 2020-05-15 14:00:00)  556623.0  9615.511719   
[2020-05-15 14:00:00, 2020-05-15 16:00:00)  796204.0  9561.266602   

symbol                                                                \
                                                   high          low   
[2020-05-15 08:00:00, 2020-05-15 10:00:00)  9638.321289  9549.190430   
[2020-05-15 10:00:00, 2020-05-15 12:00:00)  9632.041992  9565.952148   
[2020-05-15 12:00:00, 2020-05-15 14:00:00)  9631.281250  9543.962891   
[2020-05-15 14:00:00, 2020-05-15 16:00:00)  9607.120117  9542.396484   

symbol                                                          
                                                  close volume  
[2020-05-15 08:00:00, 2020-05-15 10:00:00)  9608.022461    0.0  
[2020-05-15 10:00:00, 2020-05-15 12:00:00)  9614.637695    0.0  
[2020-05-15 12:00:00, 2020-05-15 14:00:00)  9563.033203    0.0  
[2020-05-15 14:00:00, 2020-05-15 16:00:00)  9585.613281    0.0

Although `openend` was passed as "shorten" a shorter final interval was not included as it wasn't possible to evalute a half hour indice from the hourly data that's available.

**The "shortern" option is only implemented if sufficiently granular base data is available to evaluate the shorter indice.** To the contrary, the table will be returned as if `openend` were "maintain". **NOTE: No warning is raised** if `openend` is passed as "shorter" although it is not possible to evalute a shorter final indice.

(The [data availability tutorial](./data_availability.ipynb) offers a fuller explanation of data availability.)

## Daily prices

The `anchor` parameter is not relevant if `interval` is passed as a daily or monthly interval.

Indices for **intervals of multiple days** are always evaluated on a 'workback' basis, with the last session of the last indice representing the evaluated period end (recall that indices are closed "left", such that the right side of the last indice is not included to the period).

All indices contain the number of sessions represented by the `interval`. **If the number of sessions that the period covers is not a factor of the interval then the 'remainder' of sessions are lost from the start of the period**, not the end.

The following examples all request the last 4 weeks of data at various daily intervals.
Notice how the the last session is the same for all of them although the left side of the first indice varies to exclude any 'remainder'.

In [89]:
prices.get("8D", weeks=4)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-21, 2022-05-03)  288.579987  293.299988  270.000000  284.470001   
[2022-05-03, 2022-05-13)  283.959991  290.880005  250.020004  255.350006   

symbol                                 
                               volume  
[2022-04-21, 2022-05-03)  310358100.0  
[2022-05-03, 2022-05-13)  327595600.0

In [90]:
prices.get("5D", weeks=4)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-22, 2022-04-29)  281.679993  290.980011  270.000000  289.630005   
[2022-04-29, 2022-05-06)  288.609985  290.880005  274.339996  277.350006   
[2022-05-06, 2022-05-13)  274.809998  279.250000  250.020004  255.350006   

symbol                                 
                               volume  
[2022-04-22, 2022-04-29)  208727400.0  
[2022-04-29, 2022-05-06)  175014400.0  
[2022-05-06, 2022-05-13)  224757300.0

In [91]:
prices.get("4D", weeks=4)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-21, 2022-04-27)  288.579987  293.299988  270.000000  270.220001   
[2022-04-27, 2022-05-03)  282.100006  290.980011  276.220001  284.470001   
[2022-05-03, 2022-05-09)  283.959991  290.880005  271.269989  274.730011   
[2022-05-09, 2022-05-13)  270.059998  273.750000  250.020004  255.350006   

symbol                                 
                               volume  
[2022-04-21, 2022-04-27)  141057700.0  
[2022-04-27, 2022-05-03)  169300400.0  
[2022-05-03, 2022-05-09)  140586600.0  
[2022-05-09, 2022-05-13)  187009000.0

In [92]:
df = prices.get("3D", weeks=4)
df

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-19, 2022-04-22)  279.380005  293.299988  278.410004  280.809998   
[2022-04-22, 2022-04-27)  281.679993  283.200012  270.000000  270.220001   
[2022-04-27, 2022-05-02)  282.100006  290.980011  276.500000  277.519989   
[2022-05-02, 2022-05-05)  277.709991  290.880005  276.220001  289.980011   
[2022-05-05, 2022-05-10)  285.540009  286.350006  263.320007  264.579987   
[2022-05-10, 2022-05-13)  271.690002  273.750000  250.020004  255.350006   

symbol                                 
                               volume  
[2022-04-19, 2022-04-22)   74659000.0  
[2022-04-22, 2022-04-27)  111603100.0  
[2022-04-27, 2022-05-02)  134149300.0  
[2022-05-02, 2022-05-05)   94729000.0  
[2022-05-05, 2022-05-10)  128734700.0  
[2022-05-10, 2022-05-13)  139283000.0

Many of the **`pt` accessor methods** are also available for daily data.

`pt.indices_trading` queries those indices which contain only sessions, as opposed to a mix of sessions and non-session dates.

In [93]:
df.pt.indices_trading(xnys)

IntervalIndex([[2022-04-19, 2022-04-22), [2022-05-02, 2022-05-05), [2022-05-10, 2022-05-13)], dtype='interval[datetime64[ns], left]')

`pt.indices_partial_trading` queries those indices which contain a mix of sessions and non-session dates.

In [94]:
df.pt.indices_partial_trading(xnys)

IntervalIndex([[2022-04-22, 2022-04-27), [2022-04-27, 2022-05-02), [2022-05-05, 2022-05-10)], dtype='interval[datetime64[ns], left]')

`pt.indices_non_trading` queries those indices which do not contain any sessions when evaluated against the passed calendar. It will always be empty when all symbols are associated with the same calendar (although not necessarily when this is not the case). 

In [95]:
df.pt.indices_non_trading(xnys).empty

True

When **`interval` is a multiple of months** each indice will cover that multiple of calendar months.

**If the period end evaluates to 'now'** then the last indice will be considered a 'live interval' that runs to the end of the current month. The period start in this case is evaluated by working back from the period end.

In [96]:
prices.get("3M", years=1)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-06-01, 2021-09-01)  251.229996  305.839996  243.000000  301.880005   
[2021-09-01, 2021-12-01)  302.869995  349.670013  280.250000  330.589996   
[2021-12-01, 2022-03-01)  335.130005  344.299988  271.519989  298.790009   
[2022-03-01, 2022-06-01)  296.399994  315.950012  250.020004  255.350006   

symbol                                  
                                volume  
[2021-06-01, 2021-09-01)  1.472554e+09  
[2021-09-01, 2021-12-01)  1.529320e+09  
[2021-12-01, 2022-03-01)  2.270257e+09  
[2022-03-01, 2022-06-01)  1.724375e+09

If `end` is passed as the end of a month then the period end will be `end` as passed.

In [97]:
prices.get("2M", end="2021-12-31", months=6)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-07-01, 2021-09-01)  269.609985  305.839996  269.600006  301.880005   
[2021-09-01, 2021-11-01)  302.869995  332.000000  280.250000  331.619995   
[2021-11-01, 2022-01-01)  331.359985  349.670013  317.250000  336.320007   

symbol                                  
                                volume  
[2021-07-01, 2021-09-01)  9.639816e+08  
[2021-09-01, 2021-11-01)  1.019434e+09  
[2021-11-01, 2022-01-01)  1.135560e+09

If `end` is passed as any other date then the period end will be the end of the month that preceeds `end`.

In [98]:
prices.get("2M", end="2021-12-30", months=6)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-06-01, 2021-08-01)  251.229996  290.149994  243.000000  284.910004   
[2021-08-01, 2021-10-01)  286.359985  305.839996  281.619995  281.920013   
[2021-10-01, 2021-12-01)  282.119995  349.670013  280.250000  330.589996   

symbol                                  
                                volume  
[2021-06-01, 2021-08-01)  1.031245e+09  
[2021-08-01, 2021-10-01)  9.442276e+08  
[2021-10-01, 2021-12-01)  1.026401e+09

If `start` is passed (and `end` is not) and `start` is the first day of a month then the period start will evaluate to `start`.

In [99]:
prices.get("2M", start="2021-03-01", months=6)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-03-01, 2021-05-01)  235.899994  263.190002  224.259995  252.179993   
[2021-05-01, 2021-07-01)  253.399994  271.649994  238.070007  270.899994   
[2021-07-01, 2021-09-01)  269.609985  305.839996  269.600006  301.880005   

symbol                                  
                                volume  
[2021-03-01, 2021-05-01)  1.293607e+09  
[2021-05-01, 2021-07-01)  1.003657e+09  
[2021-07-01, 2021-09-01)  9.639816e+08

If `start` is passed (and `end` is not) and `start` is not the first day of a month then the period start will evaluate to the start of the month that follows `start`.

In [100]:
prices.get("2M", start="2021-03-02", months=6)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-04-01, 2021-06-01)  238.470001  263.190002  238.050003  249.679993   
[2021-06-01, 2021-08-01)  251.229996  290.149994  243.000000  284.910004   
[2021-08-01, 2021-10-01)  286.359985  305.839996  281.619995  281.920013   

symbol                                  
                                volume  
[2021-04-01, 2021-06-01)  1.063746e+09  
[2021-06-01, 2021-08-01)  1.031245e+09  
[2021-08-01, 2021-10-01)  9.442276e+08

Whenever `end` is passed, `end` determines the period end and the period start is then evaluated by working back from that period end. However, if `start` is passed, and `end` is not, then the period start is determined by `start` and the period end is evaluated by working forwards from that period start. This is shown by the following examples where the `interval` is not a factor of the period requested.

In [101]:
prices.get("2M", end="2021-09-13", months=7)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-03-01, 2021-05-01)  235.899994  263.190002  224.259995  252.179993   
[2021-05-01, 2021-07-01)  253.399994  271.649994  238.070007  270.899994   
[2021-07-01, 2021-09-01)  269.609985  305.839996  269.600006  301.880005   

symbol                                  
                                volume  
[2021-03-01, 2021-05-01)  1.293607e+09  
[2021-05-01, 2021-07-01)  1.003657e+09  
[2021-07-01, 2021-09-01)  9.639816e+08

Above the period end is evaluated as the end of the month preceeding `end` and the indices are then evaluated by working backwards. The period only covers 6 months and the excess month is excluded from the start of the data.

When `start` is passed instead of `end`...

In [102]:
prices.get("2M", start="2021-09-13", months=7)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-10-01, 2021-12-01)  282.119995  349.670013  280.250000  330.589996   
[2021-12-01, 2022-02-01)  335.130005  344.299988  276.049988  310.980011   
[2022-02-01, 2022-04-01)  310.410004  315.950012  270.000000  308.309998   

symbol                                  
                                volume  
[2021-10-01, 2021-12-01)  1.026401e+09  
[2021-12-01, 2022-02-01)  1.573206e+09  
[2022-02-01, 2022-04-01)  1.431385e+09

The period start evalutes as the start of the next month, i.e. "2021-10-13" and indices are evaluted forwards from here. The period only covers the 6 months and the extra month is excluded from the end of the data.

When both `start` and `end` are passed, the period end is evaluated and indices evaluated by working back from there.

In [103]:
prices.get("2M", "2021-02-02", "2021-12-16")

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-04-01, 2021-06-01)  238.470001  263.190002  238.050003  249.679993   
[2021-06-01, 2021-08-01)  251.229996  290.149994  243.000000  284.910004   
[2021-08-01, 2021-10-01)  286.359985  305.839996  281.619995  281.920013   
[2021-10-01, 2021-12-01)  282.119995  349.670013  280.250000  330.589996   

symbol                                  
                                volume  
[2021-04-01, 2021-06-01)  1.063746e+09  
[2021-06-01, 2021-08-01)  1.031245e+09  
[2021-08-01, 2021-10-01)  9.442276e+08  
[2021-10-01, 2021-12-01)  1.026401e+09

Notice that `start` is nearly 2 months away from the actual period start. This shows how the period end is evalauted and indices are then evaluated working backwards. To include another interval at the start would result in 2021-02-01 being included, which goes against the Golden Rule that the evaluated period can not include any period before `start` or after `end`.

### Daily prices with multiple calendars

**Intervals as a multiple of days** are evalauted in the same manner whether symbols are all associated with the same calendar or not. In both cases indices are evaluted to all contain the same number of sessions. When symbols are not all associated with the same calendar the sessions are evaluated in accordance with the calendar associated with the lead symbol.

The difference can be illustrated by considering prices covering Microsoft, which usually trades Mon-Fri, and Bitcoin, which trades everyday.

In [104]:
prices_us_247 = PricesYahoo("MSFT, BTC-USD")
prices_us_247.get("3D", weeks=4, lead_symbol="MSFT")

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-19, 2022-04-22)  279.380005  293.299988  278.410004  280.809998   
[2022-04-22, 2022-04-27)  281.679993  283.200012  270.000000  270.220001   
[2022-04-27, 2022-05-02)  282.100006  290.980011  276.500000  277.519989   
[2022-05-02, 2022-05-05)  277.709991  290.880005  276.220001  289.980011   
[2022-05-05, 2022-05-10)  285.540009  286.350006  263.320007  264.579987   
[2022-05-10, 2022-05-13)  271.690002  273.750000  250.020004  255.350006   

symbol                                      BTC-USD                \
                               volume          open          high   
[2022-04-19, 2022-04-22)   74659000.0  40828.175781  42893.582031   
[2022-04-22, 2022-04-27)  111603100.0  40525.863281  40777.757812   
[2022-04-27, 2022-05-02)  134149300.0  38120.300781  40269.464844   
[2022-05-02, 2022-05-05)   94729000.0  38472.187500  39902.949219   
[2022-05-05, 2022-05-10)  128734700.0  39695.746094  39789.281250   
[2022-05-10, 2022-05-13)  139283000.0  30273.654297  32596.308594   

symbol                                                              
                                   low         close        volume  
[2022-04-19, 2022-04-22)  40063.828125  40527.363281  8.849553e+10  
[2022-04-22, 2022-04-27)  37884.984375  38117.460938  1.321290e+11  
[2022-04-27, 2022-05-02)  37585.789062  38469.093750  1.466662e+11  
[2022-05-02, 2022-05-05)  37585.621094  39698.371094  9.700399e+10  
[2022-05-05, 2022-05-10)  30296.953125  30296.953125  2.053963e+11  
[2022-05-10, 2022-05-13)  26350.490234  29047.751953  1.971891e+11

When MSFT is the lead symbol (above) the index is the same as for prices requested from the earlier Prices class that included only MSFT...

In [105]:
prices.get("3D", weeks=4)

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-19, 2022-04-22)  279.380005  293.299988  278.410004  280.809998   
[2022-04-22, 2022-04-27)  281.679993  283.200012  270.000000  270.220001   
[2022-04-27, 2022-05-02)  282.100006  290.980011  276.500000  277.519989   
[2022-05-02, 2022-05-05)  277.709991  290.880005  276.220001  289.980011   
[2022-05-05, 2022-05-10)  285.540009  286.350006  263.320007  264.579987   
[2022-05-10, 2022-05-13)  271.690002  273.750000  250.020004  255.350006   

symbol                                 
                               volume  
[2022-04-19, 2022-04-22)   74659000.0  
[2022-04-22, 2022-04-27)  111603100.0  
[2022-04-27, 2022-05-02)  134149300.0  
[2022-05-02, 2022-05-05)   94729000.0  
[2022-05-05, 2022-05-10)  128734700.0  
[2022-05-10, 2022-05-13)  139283000.0

However, if the lead symbol is changed to BTC-USD then there will be more indices as weekend sessions will now contribute to the "3D" interval.

In [106]:
df = prices_us_247.get("3D", weeks=4, lead_symbol="BTC-USD")
df

symbol                          MSFT                                      \
                                open        high         low       close   
[2022-04-17, 2022-04-20)  278.910004  286.170013  278.339996  285.299988   
[2022-04-20, 2022-04-23)  289.399994  293.299988  273.380005  274.029999   
[2022-04-23, 2022-04-26)  273.290009  281.109985  270.769989  280.720001   
[2022-04-26, 2022-04-29)  277.500000  290.980011  270.000000  289.630005   
[2022-04-29, 2022-05-02)  288.609985  289.880005  276.500000  277.519989   
[2022-05-02, 2022-05-05)  277.709991  290.880005  276.220001  289.980011   
[2022-05-05, 2022-05-08)  285.540009  286.350006  271.269989  274.730011   
[2022-05-08, 2022-05-11)  270.059998  273.750000  263.320007  269.500000   
[2022-05-11, 2022-05-14)  265.679993  271.359985  250.020004  255.350006   

symbol                                      BTC-USD                \
                               volume          open          high   
[2022-04-17, 2022-04-20)   43075700.0  40417.777344  41672.960938   
[2022-04-20, 2022-04-23)   81767100.0  41501.746094  42893.582031   
[2022-04-23, 2022-04-26)   35678900.0  39738.722656  40491.753906   
[2022-04-26, 2022-04-29)  143642700.0  40448.421875  40713.890625   
[2022-04-29, 2022-05-02)   37025000.0  39768.617188  39887.269531   
[2022-05-02, 2022-05-05)   94729000.0  38472.187500  39902.949219   
[2022-05-05, 2022-05-08)   81008700.0  39695.746094  39789.281250   
[2022-05-08, 2022-05-11)   87062400.0  35502.941406  35502.941406   
[2022-05-11, 2022-05-14)   99946600.0  31016.183594  32013.402344   

symbol                                                              
                                   low         close        volume  
[2022-04-17, 2022-04-20)  38696.191406  41502.750000  7.809602e+10  
[2022-04-20, 2022-04-23)  39315.417969  39740.320312  9.120404e+10  
[2022-04-23, 2022-04-26)  38338.378906  40458.308594  6.954815e+10  
[2022-04-26, 2022-04-29)  37884.984375  39773.828125  9.945381e+10  
[2022-04-29, 2022-05-02)  37585.789062  38469.093750  8.178147e+10  
[2022-05-02, 2022-05-05)  37585.621094  39698.371094  9.700399e+10  
[2022-05-05, 2022-05-08)  34940.824219  35501.953125  1.052777e+11  
[2022-05-08, 2022-05-11)  29944.802734  31022.906250  1.599296e+11  
[2022-05-11, 2022-05-14)  26350.490234  30514.349609  1.922683e+11

The `.pt.indices_partial_trading` method shows that there are many indices that cover dates that were not xnys sessions.

In [107]:
df.pt.indices_partial_trading(xnys)

IntervalIndex([[2022-04-17, 2022-04-20), [2022-04-23, 2022-04-26), [2022-04-29, 2022-05-02), [2022-05-05, 2022-05-08), [2022-05-08, 2022-05-11)], dtype='interval[datetime64[ns], left]')

Although there are no such indices for the 24/7 trading bitcoin...

In [108]:
x247 = prices_us_247.calendars["BTC-USD"]
df.pt.indices_partial_trading(x247)

IntervalIndex([], dtype='interval[datetime64[ns], left]')

Indices for **intervals that are a multiple of months** are evaluated in the same way whether symbols are all associated with the same calendar or not.

In [109]:
prices_us_lon.get("3M", years=1)

symbol                          MSFT                                      \
                                open        high         low       close   
[2021-06-01, 2021-09-01)  251.229996  305.839996  243.000000  301.880005   
[2021-09-01, 2021-12-01)  302.869995  349.670013  280.250000  330.589996   
[2021-12-01, 2022-03-01)  335.130005  344.299988  271.519989  298.790009   
[2022-03-01, 2022-06-01)  296.399994  315.950012  250.020004  255.350006   

symbol                                   AZN.L                                \
                                volume    open     high          low   close   
[2021-06-01, 2021-09-01)  1.472554e+09  8095.0   8811.0  7870.000000  8514.0   
[2021-09-01, 2021-12-01)  1.529320e+09  8535.0   9523.0  8029.000000  8276.0   
[2021-12-01, 2022-03-01)  2.270257e+09  8303.0   9175.0  8090.319824  9059.0   
[2022-03-01, 2022-06-01)  1.724375e+09  9195.0  11000.0  8326.000000  9965.0   

symbol                                 
                               volume  
[2021-06-01, 2021-09-01)  223099794.0  
[2021-09-01, 2021-12-01)  134350339.0  
[2021-12-01, 2022-03-01)  140648946.0  
[2022-03-01, 2022-06-01)  144222024.0

## Takeaways

If [`anchor` is **"open"**](#anchor-"open") (the default) then by default all indices will be of length `interval`.
* If the last indice of a (sub)session does not align with the (sub)session close then that indice will include a non-trading period (unless calendar is 24h).
* If the indice of a (sub)session [overlaps](#Indices-overlapping-next-session) the first indice of the next sub(session) then the right side of that indice will be curtailed to the left side of the next indice. In this case a Warning is raised to advise that the intervals are not regular.
* The [`force` option](#force) will curtail indices that include a single non-trading period so that they only cover the period during which at least one symbol was trading. 
* If symbols are associated with [multiple calendars](#Multiple-calendars) prices will be anchored to the open of the `lead_symbol`.
* There are [some cases](#Calendars-with-breaks) where indices associated with an afternoon subsession are necessarily anchored to the morning open.
* The [`openend` option](#Force-to-period-end-with-openend) determines how the final indice should be defined when the period end is an unaligned session close.
    * "maintain" ensures the final indice has length `interval`. The final indice will be the indice that contains the session close only if no symbol trades in the period between the session close and the right of this indice, to the contrary the final indice will be the prior indice.
    * "shorten" will result in the final indice being shorter than `interval` and aligning with the session close (albeit [only if underlying data allows](#openend)).

Passing [`anchor` as **"workback"**](#anchor-"workback") will anchor the right of the last indice on the evaluated period end. Indices are then evaluated by working back the number of minutes corresponding with `interval`.
* All indices will cover `interval` number of trading minutes.
* Non-trading periods between (sub)sessions will be included to indices whenever indices don't happen to neatly align with session opens.

There are a host of [`.pt` accessor methods](#Interrogating-indices-with-.pt-accessor), to interrogate the likes of:
* indices' trading status (trading, partial-trading, non-trading)
* the number of trading minutes associated with indices
* indices length

Many [`.pt` accessor methods take a calendar argument](#.pt-accessor-methods) to evaluate the return for a specific calendar.